## Loading Data and Parsing Events

In [ ]:
from matplotlib import pyplot as plt
import numpy as np
import PyPore
from PyPore.DataTypes import *
import seaborn as sns
import sys
import os
from Dimer_Analysis import TraceFile
import Dimer_Analysis 
fname=os.path.join("D:\\ProteinData\Meni\\alphaHL_ GFP10D_1M KCl_2M GdnHCl_PH7.5  traces 10kHz filter\\175mV","B091520_005_trace_10kHz.bin")

def prepareFiles(directory,inputDict,metaOutput="metadata.json",verbose=False ,acceptedExts={".bin",".opt",".abf"},meta_format="json"):
    for root,dirs,files in os.walk(directory,topdown=True):
        for name in files:
            for key,value in inputDict.items():
                if root.find(key) is not -1 and root.find("175") is not -1:
                    if os.path.splitext(name)[1]==".bin":
                        value.append(os.path.join(root,name))
PyPore.DataTypes.__file__

In [ ]:
###Filter Derivative Parsing Algorithm###
class MyFilterDerivativeSegmenter( parser ):
    '''
    This parser will segment an event using a filter-derivative method. It will
    first apply a bessel filter at a certain cutoff to the current, then it will
    take the derivative of that, and segment when the derivative passes a
    threshold.
    '''

    def __init__( self, low_threshold=0.025, high_threshold=0.12, cutoff_freq=10000.,
        sampling_freq=1.e5 ,openholder=None,openlimits=[0.8,1.2]):
        self.low_threshold = low_threshold
        self.high_threshold = high_threshold
        self.cutoff_freq = cutoff_freq
        self.sampling_freq = sampling_freq
        self.openholder=openholder
        self.openlimits=openlimits

    def parse( self, current ):
        '''
        Apply the filter-derivative method to filter the ionic current.
        '''
        binCount=100
        edges=np.histogram(current,np.histogram_bin_edges(current,binCount))
        currentCpy=np.array(current,copy=True)
        currentCpy=currentCpy[((edges[1][-1]/2)<currentCpy)]
        upperEdges=np.histogram(currentCpy,np.histogram_bin_edges(currentCpy,int(binCount/2)))
        i_peak=np.argmax(upperEdges[0])
        I_0_MLE=np.mean(upperEdges[1][i_peak-1:i_peak+3])
        # Filter the current using a first order Bessel filter twice, one in
        # both directions to preserve phase
        from scipy import signal
        nyquist = self.sampling_freq / 2.
        b, a = signal.bessel( 1, self.cutoff_freq / nyquist, btype='low', analog=0, output='ba' )
        filtered_current = signal.filtfilt( b, a, np.array( current ).copy() )
        #print(filtered_current)
        #print(current)
        # Take the derivative
        deriv = np.abs( np.diff( filtered_current ) )
        #purederiv = np.diff( filtered_current )
        # Find the edges of the blocks which fulfill pass the lower threshold
        blocks = np.where( deriv > self.low_threshold*I_0_MLE, 1, 0 )
        block_edges = np.abs( np.diff( blocks ) )
        tics = np.where( block_edges == 1 )[0] + 1 

        # Split points are points in the each block which pass the high
        # threshold, with a maximum of one per block 
        split_points = [0] 

        for start, end in zip( tics[:-1:2], tics[1::2] ): # For all pairs of edges for a block..
            segment = deriv[ start:end ] # Save all derivatives in that block to a segment
            segmentCurrent= filtered_current[start:end]
            #print(np.amax(segmentCurrent), end=" ")
            if (np.argmax( segment ) > self.high_threshold*I_0_MLE) and (np.amax(segmentCurrent)>I_0_MLE*0.8): # If the maximum derivative in that block is above a threshold..
                split_points = np.concatenate( ( split_points, [ start, end ] ) ) # Save the edges of the segment 
                # Now you have the edges of all transitions saved, and so the states are the current between these transitions
        tics = np.concatenate( ( split_points, [ current.shape[0] ] ) )
        #tics = map( int, tics )
        #print(tics)
        events=[ Segment( current=current[ tics[i]:tics[i+1] ], start=tics[i],duration=tics[i+1]-tics[i] ) 
                    for i in range( 0, len(tics)-1, 2 ) ]
        currents=[]
        currents = [event.current for event in events if event.min>I_0_MLE*self.openlimits[0] and event.max<I_0_MLE*self.openlimits[1]]
        conCurrents=np.hstack(currents)
        I_0 = np.mean(conCurrents)#find I_0 for the trace
        for event in events:
            event.I_0=I_0
        if self.openholder is not None: 
            self.openholder.append(conCurrents)
        return [event for event in events if event.min< I_0_MLE*0.30 and event.max<I_0_MLE*0.5]

    def set_params( self ):
        self.low_thresh = float( self.lowThreshInput.text() )
        self.high_thresh = float( self.highThreshInput.text() )


In [ ]:
###Updated so that we incorporate the real Fourier coefficients in the feature space####
#experimentfiles={"_ GFP10D":[],"_ MBP10D":[]}
#experimentfiles={"_ GFP10D":[],"_ MBP10D":[],"_ MBP+GFP10D":[]}#,"_ MBPMBP10D":[],"_ MBP Nterm":[]}
experimentfiles={"_ GFP10D":[],"_ MBP10D":[],"_ MBPMBP10D":[],"_ MBP Nterm":[]}
#experimentfiles={"_ MBP10D":[],"_ MBPMBP10D":[]}
#experimentfiles={"_ GFP10D":[],"_ MBP10D":[],"_ MBPMBP10D":[]}
#experimentfiles={"_ GFP10D":[],"_ GFP 5minDye 10D":[],"_ GFP Dye 10D":[]}
#experimentfiles={"_ MBP10D":[],"_ MBP Nterm":[]}

#experimentfiles={"_ MBP10D":[]}

#######################################################################
# experimentfiles={"_ GFP10D":[],"_ MBP10D":[],
#                  "_ MBP+GFP10D_20_80":[], "_ MBP+GFP10D_50_50":[], 
#                  "_ MBPMBP10D":[], "_ MBP Nterm":[], "_ MBP+GFP10D_40_60":[],
#                  "_ MBP+GFP10D_60_40":[], "_ MBP+GFP10D_80_20":[]}
#######################################################################


#experimentfiles={"_ GFP10D":[],"_ MBP10D":[],
#                 "_ MBP+GFP10D_20_80":[], "_ MBP+GFP10D_50_50":[], "_ MBP+GFP10D_40_60":[],
#                 "_ MBP+GFP10D_60_40":[], "_ MBP+GFP10D_80_20":[]}


prepareFiles("D:\\ProteinData\\Meni",experimentfiles) #Directory containing all the experimental files
experimentfilesmod={}
experiments={}

for key,value in experimentfiles.items():
    print(key[2:]," ")
    experimentfilesmod[key[2:]] = value
    experiments[key[2:]]=[]
    for filename in value:
        print(" ",filename)

print (experiments)

import time
t=time.time()
for key,filenames in experimentfilesmod.items():
    for filename in filenames:
        experiments[key].append(TraceFile(filename))
        experiments[key][-1].parse(parser=MyFilterDerivativeSegmenter(sampling_freq=experiments[key][-1].second,cutoff_freq=5000,low_threshold=0.010,high_threshold=0.015)) 
        experiments[key][-1].splitEvents(segmentCount=10)
        experiments[key][-1].quantileSignal() #Taking the first quartile, median, and third quartile of each segment
print ("elapsed time:",time.time()-t)

## Load event features into dataframe

In [ ]:
import pandas as pd

#print(experiments)

eventsDf=pd.DataFrame(columns=["type","file","I_0","mean","mean/I_0","std","std/I_0","duration"])

for key,files in experiments.items():
    print (key)
    for file in files:
        for event in file.events:
            #print(event)
            d={"type":  key,
                             "file":  file.filename,
                             "I_0":   event.I_0,
                             "mean":  event.mean,
                             "mean/I_0":event.mean/event.I_0,
                             "std":event.std,
                             "std/I_0":event.std/event.I_0,
                             "duration":event.duration,
                             "event":event,
                             "event_start":event.start,
                             "event_end":event.end,
                             "Imin":event.min,
                             "Imax":event.max,
                             "Imin/I_0":event.min/event.I_0,
                             "Imax/I_0":event.max/event.I_0,
                             "min/std":event.min/event.std,
                             "min/mean":event.min/event.mean,
                             "min/duration":event.min/event.duration,
                             "min/max":event.min/event.max,
                             "duration/std":event.duration/event.std,
                             "max/std":event.max/event.std,
                             "mean/std":event.mean/event.std,
                             "Blockage":round((event.I_0-event.mean)/event.I_0, 3)}#,
                             #"segsmean/I_0":[seg.mean/event.I_0 for seg in event.segments],
                             #"segsstd/I_0":[seg.std/event.I_0 for seg in event.segments]}
            
            for i in range (len(event.segments)):
                d["segment{}_mean".format(i)]=event.segments[i].mean/event.I_0
            for i in range (len(event.segments)):
                d["segment{}_std".format(i)]=event.segments[i].std/event.I_0
                
            for i in range (len(event.segments)):
                d["segment{}_min".format(i)]=event.segments[i].min/event.I_0
            for i in range (len(event.segments)):
                d["segment{}_max".format(i)]=event.segments[i].max/event.I_0
                
            for i in range (len(event.segments)):
                d["segment{}_first_quartile".format(i)]=event.first_quartiles[i]/event.I_0
                d["segment{}_median".format(i)]=event.medians[i]/event.I_0
                d["segment{}_third_quartile".format(i)]=event.third_quartiles[i]/event.I_0

            ##########################################
            eventsDf=eventsDf.append(d,ignore_index=True)


print(eventsDf)


# Machine Learning
## Gradient Boosting Classifier

The following scripts handle GBC model training and testing

In [ ]:
####Checking Accuracy of different ML models#####
import sklearn as sk
print('The scikit-learn version is {}.'.format(sk.__version__))
from sklearn.model_selection import train_test_split
import seaborn as sns
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier,ExtraTreesClassifier,AdaBoostClassifier,GradientBoostingClassifier
#from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report, confusion_matrix

#print(len(eventsDf.loc[eventsDf["type"]=="MBP10D"]))
#nomixed=eventsDf.loc[eventsDf["type"]=="GFP10D"]
#nomixed=eventsDf.loc[eventsDf['type']!="MBP Nterm"].loc[eventsDf['type']!="GFP10D"]
#nomixed=eventsDf.loc[eventsDf['type']!="MBP Nterm"]

nomixed=eventsDf

prior_NA = len(nomixed)
nomixed=nomixed.loc[nomixed['duration']>300e-6].loc[nomixed['duration']<20e-3] #minimum of 125 samples and maximum of 5000 samples
#nomixed=nomixed.loc[nomixed['duration']>2e-3].loc[nomixed['duration']<20e-3] #minimum of 500 samples and maximum of 5000 samples

#nomixed=nomixed.loc[nomixed['duration']>900e-6]

post_NA = len(nomixed)
removed_NA_df = nomixed.dropna()

print(f"Size of dataframe prior to filtering by duration: {prior_NA}")
print(f"Size of dataframe after filtering by duration: {post_NA}")
print(f"Size of dataframe after  filtering by duration and dropping NaNs from dataframe: {len(removed_NA_df)}")

#print(f"How many events are in each file before any removal: \n{nomixed['file'].value_counts()}")


#Resampling to make MBP10D and GFP10D the same size for training and testing
#nomixed = nomixed.mask(nomixed.eq('None')).dropna()

#n = len(nomixed.loc[nomixed['type']=='MBP10D'])
n = len(nomixed.loc[nomixed['type']=='GFP10D'])
#n = len(nomixed.loc[nomixed['type']=='MBP Nterm'])
#n = len(nomixed.loc[nomixed['type']=='MBPMBP10D'])
seedVal = random.randint(0, 200)
#################################################################################
resamp_MBP10D = nomixed.loc[nomixed["type"]=="MBP10D"].sample(n, random_state=seedVal)
resamp_GFP10D = nomixed.loc[nomixed["type"]=="GFP10D"].sample(n, random_state=seedVal)
noMixed = pd.concat([resamp_MBP10D, resamp_GFP10D])
#################################################################################
#file_remove = "\B062622_000_trace.bin"
#nomixed = nomixed.loc[nomixed["file"]!="\B062622"].loc[nomixed["file"]!="\B062627"]
#df[df["A"].str.contains("Hello|Britain")]
#resamp_MBP10D=nomixed.loc[nomixed['type']=='MBP10D']
#print(len(resamp_MBP10D))
#resamp_MBP10D=resamp_MBP10D[resamp_MBP10D["file"].str.contains("\B081920")==True]
#print(len(resamp_MBP10D))

#resamp_MBP10D=nomixed.loc[nomixed['type']=='MBP10D'].sample(n, random_state=seedVal)
#resamp_MBPMBP10D = nomixed.loc[nomixed["type"]=="MBPMBP10D"].sample(n, random_state=seedVal)
#noMixed = pd.concat([resamp_MBP10D, resamp_MBPMBP10D])

#################################################################################

#resamp_NtermMPB10D = nomixed.loc[nomixed["type"]=="MBP Nterm"].sample(n, random_state=seedVal)
#resamp_MBPMBP10D = nomixed.loc[nomixed["type"]=="MBPMBP10D"].sample(n, random_state=seedVal)
#noMixed = pd.concat([resamp_NtermMPB10D, resamp_MBPMBP10D])

#################################################################################

#resamp_MBP10D = nomixed.loc[nomixed["type"]=="MBP10D"].sample(n, random_state=seedVal)
#resamp_NtermMPB10D = nomixed.loc[nomixed["type"]=="MBP Nterm"].sample(n, random_state=seedVal)
#noMixed = pd.concat([resamp_MBP10D, resamp_NtermMPB10D])

#################################################################################
#################################################################################
#resamp_NtermMPB10D = nomixed.loc[nomixed["type"]=="MBP Nterm"].sample(n, random_state=seedVal)
#resamp_GFP10D = nomixed.loc[nomixed["type"]=="GFP10D"].sample(n, random_state=seedVal)
#noMixed = pd.concat([resamp_NtermMPB10D, resamp_GFP10D])
#################################################################################
#################################################################################

#resamp_MBPMBP10D = nomixed.loc[nomixed["type"]=="MBPMBP10D"].sample(n, random_state=1)
#resamp_GFP10D = nomixed.loc[nomixed["type"]=="GFP10D"].sample(n, random_state=seedVal)
#noMixed = pd.concat([resamp_MBPMBP10D, resamp_GFP10D])

#################################################################################

#resamp_MBP10D = nomixed.loc[nomixed["type"]=="MBP10D"].sample(n, random_state=10)
#resamp_MBPMBP10D = nomixed.loc[nomixed["type"]=="MBPMBP10D"].sample(n, random_state=10)
#resamp_GFP10D = nomixed.loc[nomixed["type"]=="GFP10D"].sample(n, random_state=10)
#noMixed = pd.concat([resamp_MBP10D, resamp_MBPMBP10D, resamp_GFP10D])

#################################################################################

#################################################################################

#resamp_MBP10D = nomixed.loc[nomixed["type"]=="MBP10D"].sample(n, random_state=seedVal)
#resamp_MBPMBP10D = nomixed.loc[nomixed["type"]=="MBPMBP10D"].sample(n, random_state=seedVal)
#resamp_NtermMPB10D = nomixed.loc[nomixed["type"]=="MBP Nterm"].sample(n, random_state=seedVal)
#noMixed = pd.concat([resamp_MBP10D, resamp_MBPMBP10D, resamp_NtermMPB10D])

#################################################################################
#resamp_MBP10D = nomixed.loc[nomixed["type"]=="MBP10D"].sample(n, random_state=1)
#resamp_GFP10D = nomixed.loc[nomixed["type"]=="GFP10D"].sample(n, random_state=1)
#resamp_MBPMBP10D = nomixed.loc[nomixed["type"]=="MBPMBP10D"].sample(n, random_state=1)
#resamp_NtermMPB10D = nomixed.loc[nomixed["type"]=="MBP Nterm"].sample(n, random_state=1)
#noMixed = pd.concat([resamp_MBP10D, resamp_GFP10D, resamp_MBPMBP10D, resamp_NtermMPB10D])
#################################################################################


#Show us how many pure samples of GFPD10 and MBPD10 events we have available for this ML analysis
#print(f"How many MBPD10 and GFPD10: \n{noMixed['type'].value_counts()}")
#print(" ... \n")

print(f"How many MBPD10 and GFPD10 and Dimer_MBPD10: \n{nomixed['type'].value_counts()}")
print(" ... \n")

#print(f"How many MBPD10 and GFPD10 and NtermMBPD10 and Dimer_MBPD10: \n{nomixed['type'].value_counts()}")
#print(" ... \n")

#print(f"How many MBPD10 and Dimer_MBPD10: \n{nomixed['type'].value_counts()}")
#print(" ... \n")

#########################################################
feature_space = []
remove_fc = "FC"
for (columnName, columnData) in noMixed.iteritems():
    if remove_fc in columnName:
        pass
    else:
        feature_space.append(columnName)

noMixed = noMixed.loc[:, feature_space]

#########################################################

#Feature space preparation
#x=nomixed.drop(columns=['event','file','I_0','Imax', 'mean', 'std', 'event_start', 'event_end', 'fc_0'])
x=noMixed.drop(columns=['event','file','I_0','Imax', 'mean', 'std', 'event_start', 'event_end'])#, 'fc_0'])
print(f"How many samples in the training and testing combined dataframe: \n{x['type'].value_counts()}")
print(" ... \n")


#More features are removed
x=x.drop(columns=['type','Blockage','mean/I_0', 'duration', 'std/I_0', 'Imin', 'max/std', 'min/duration', 'duration/std'])#'Imax/I_0','min/max','Imin/I_0','min/mean','min/std', 'mean/std'])
x=x.drop(columns=['min/max', 'min/mean', 'min/std', 'mean/std', 'Imin/I_0', 'Imax/I_0'])#'Imax/I_0','min/max','Imin/I_0','min/mean','min/std', 'mean/std'])


#training_features = ['min/std','Imax/I_0','sm1','Imin/I_0','min/mean','sm0','ss0','sm1','ss1',
#                     'sm2','ss2','seg0_fc4','seg1_fc4','seg2_fc4','sm4','ss4','sm5','ss5',
#                     'sm6','ss6','seg4_fc4','seg5_fc4','seg6_fc4'] #subsequences Mean
#x = x.loc[:, training_features]

feature_space = []
remove_fc0 = "_fc0"
for (columnName, columnData) in x.iteritems():
    if remove_fc0 in columnName:
        pass
    else:
        feature_space.append(columnName)

x = x.loc[:, feature_space]


########################################################################
#feature_space_updated = []
#extra_removal = ["ss8", "sm3", "sm4", "ss9", "sm9", "sm6", "ss7", "ss1"]        
#x = x.drop(columns=extra_removal)
########################################################################
print(x)
###Remove all segment FCs or Fourier coefficients###
# feature_space = []
# remove_fc0 = "fc"
# for (columnName, columnData) in x.iteritems():
#     if remove_fc0 in columnName:
#         pass
#     else:
#         feature_space.append(columnName)

# x = x.loc[:, feature_space]





######################
y=noMixed['type']
#y=nomixed['type']
######################

features = []
for (columnName, columnData) in x.iteritems():
    features.append(columnName)

print(f"The following list contains the feature space used for training the model: \n{features}")

seedVal = random.randint(0, 200)
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.2, random_state=seedVal) #2 #40 (123 seed was good for all four analytes)
scaler = sk.preprocessing.StandardScaler()
X_train = scaler.fit_transform( x_train )
X_test = scaler.transform( x_test )
print("...")



gbc = GradientBoostingClassifier(n_estimators=200,max_depth=5)


learning_rate = 0.05
max_feature = 10
max_depth = 3
min_samples_split = 0.35
min_samples_leaf = 0.35
#from xgboost import XGBClassifier
gbc = GradientBoostingClassifier(n_estimators=500, 
                                 learning_rate=learning_rate, 
                                 max_depth=max_depth, 
                                 max_features=max_feature, 
                                 min_samples_split=min_samples_split,
                                 min_samples_leaf=min_samples_leaf)



#rfc = RandomForestClassifier(n_estimators=200, max_depth=5)#, max_features=None)
#logisticRegr = LogisticRegression(solver = 'lbfgs')
#knn = KNeighborsClassifier(n_neighbors=5)
#clf = SVC(decision_function_shape='ovr',probability=True,kernel='rbf', 
#          C=200,verbose=True,gamma='scale', class_weight = None)

#gbc.fit(x_train, y_train)
gbc.fit(X_train, y_train)
#clf.fit(x_train, y_train)

y_pred=gbc.predict(X_test)
#y_pred=clf.predict(x_test)

#print(recall_score(y_test, y_pred, pos_label="MBP10D"))
print('...')
print('Size of training set {}.'.format(len(x_train)))
print('Size of testing set {}.'.format(len(x_test)))
print('Classification accuracy of GBC model: {} and the training test split seed value: {}'.format(accuracy_score(y_test,y_pred), seedVal))
#print('Classification accuracy of SVM model: {}'.format(accuracy_score(y_test,y_pred)))
#print(classification_report(y_test, y_pred))

conf_mat = confusion_matrix(y_true=y_test, y_pred=y_pred)#, normalize='true' )
print('Confusion matrix:\n', conf_mat)

feature_imp = pd.Series(gbc.feature_importances_,index=features).sort_values(ascending=False)

with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
    print(feature_imp)



## Average meature importance over multiple model generations

In [ ]:
###Looping Model multiple times to get an average results on feature importance###

#type_list = ["MBP10D", "GFP10D"]
#type_list = ["MBP10D", "MBP Nterm"]
#type_list = ["MBP10D", "MBPMBP10D"]
#type_list = ["MBP Nterm", "MBPMBP10D"]
type_list = ["MBP10D", "MBPMBP10D", "MBP Nterm"]
#type_list = ["MBPMBP10D", "GFP10D"]
limiting_sample_size = n

loops = 10

d = {}
for loop in range(loops):
    seedVal = random.randint(0, 200)
    updated_df_list = []
    for label in type_list:
        df = nomixed.loc[nomixed["type"]==label].sample(limiting_sample_size, random_state=seedVal)
        updated_df_list.append(df)
    noMixed = pd.concat(updated_df_list)
    x=noMixed.drop(columns=['event','file','I_0','Imax', 'mean', 'std', 'event_start', 'event_end'])
    print(f"How many samples in the training and testing combined dataframe: \n{x['type'].value_counts()}")
    print(" ... \n")
    x=x.drop(columns=['type','Blockage','mean/I_0', 'duration', 'std/I_0', 'Imin', 'max/std', 'min/duration', 'duration/std'])
    x=x.drop(columns=['min/max', 'min/mean', 'min/std', 'mean/std', 'Imin/I_0', 'Imax/I_0'])
    x = x.loc[:, feature_space]
    y=noMixed['type']
    
    x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.2, random_state=seedVal) #2 #40 (123 seed was good for all four analytes)
    scaler = sk.preprocessing.StandardScaler()
    X_train = scaler.fit_transform( x_train )
    X_test = scaler.transform( x_test )
    print("...")
    gbc = GradientBoostingClassifier(n_estimators=200,max_depth=5)
    #gbc.fit(x_train, y_train)
    gbc.fit(X_train, y_train)
    #clf.fit(x_train, y_train)
    y_pred=gbc.predict(X_test)
    #y_pred=clf.predict(x_test)
    #print(recall_score(y_test, y_pred, pos_label="MBP10D"))
    print('...')
    print('Size of training set {}.'.format(len(x_train)))
    print('Size of testing set {}.'.format(len(x_test)))
    print('Classification accuracy of GBC model: {} and the training test split seed value: {}'.format(accuracy_score(y_test,y_pred), seedVal))
    #print(classification_report(y_test, y_pred))
    conf_mat = confusion_matrix(y_true=y_test, y_pred=y_pred)#, normalize='true' )
    print('Confusion matrix:\n', conf_mat)
    feature_imp = pd.Series(gbc.feature_importances_,index=features).sort_values(ascending=False)
    
    feature_imp_df = pd.DataFrame(gbc.feature_importances_,index=features)#.sort_values(ascending=False)
    feature_imp_df.columns = ['Weight']
    feature_imp_df['Feature'] = feature_imp_df.index
    feature_imp_df = feature_imp_df.reset_index(drop=True)
    #print(feature_imp_df)
    #for seg in range(0, 10):
    if loop == 0:
        for seg in range(0, 10):
            weight_list = []
            segnum = str(seg)
            for idx, feature in enumerate(feature_imp_df["Feature"]):
                if segnum in feature:
                    value = feature_imp_df["Weight"][idx]
                    weight_list.append(round(value,2))
            #print(weight_list)
            d[f"Segment {seg+1}"] = weight_list
    else:
        for seg in range(0, 10):
            weight_list = []
            segnum = str(seg)
            for idx, feature in enumerate(feature_imp_df["Feature"]):
                if segnum in feature:
                    value = feature_imp_df["Weight"][idx]
                    weight_list.append(round(value,5))
            #print(weight_list)
            d[f"Segment {seg+1}"] = [a + b for a, b in zip(d[f"Segment {seg+1}"], weight_list)]
        
for seg in range(0, 10):
    print(d[f"Segment {seg+1}"])
    d[f"Segment {seg+1}"] = [round(x / loops, 2) for x in d[f"Segment {seg+1}"]]

#l1 = [90, 7, 30, 6]
#l2 = [8,  2, 40, 5]

#new = [a+b for a, b in zip(l1, l2)]
#d = {}

training_features = ["first quartile", "maximum", "mean", "median", "minimum", "standard deviation", "third quartile"]
segs_df = pd.DataFrame(d, index=training_features)

segs_df = segs_df*100
print(segs_df)


sns.set(font_scale=2)
x_axis_labels = ["S1", "S2", "S3", "S4", "S5", "S6", "S7", "S8", "S9", "S10"] # labels for x-axis
#y_axis_labels = [11,22,33,44,55,66,77,88,99,101,111,121] # labels for y-axis
vmin = 0
vmax = 25
plt.figure(figsize=(10, 10), dpi = 300)
h = sns.heatmap(segs_df,
            xticklabels=x_axis_labels,
            annot=True,
            annot_kws={"fontsize":25},
            vmin=vmin, 
            vmax=vmax, 
            cmap='Reds',
            yticklabels=False)
#plt.savefig('Features_HeatMap_MBPD10_vs_D10MBP_vs_diMBPD10_Updated.png', transparent=True)
#plt.savefig('Features_HeatMap_MBPD10_vs_D10MBP_vs_diMBPD10_Updated.svg', transparent=True)

#plt.savefig('Features_HeatMap_MBPD10_vs_GFPD10_Updated.png', transparent=True)
#plt.savefig('Features_HeatMap_MBPD10_vs_GFPD10_Updated.svg', transparent=True)

## Generate multiple GBC models and invoke on different mixture experiments

In [ ]:
####Generate mutliple iterations of the model####
from sklearn.metrics import plot_confusion_matrix
from matplotlib import colors
%matplotlib inline

#nomixed=nomixed.loc[nomixed['duration']>500e-6].loc[nomixed['duration']<20e-3] #minimum of 125 samples and maximum of 5000 samples

calls_dict = {'GFP10D': [], 'MBP10D': []}

#############################################################################
####50 versus 50 ratio file removal####
#filter_file = "B092" #pre revision experiment 1 50 versus 50 data
#filter_file = "B0706" #post revision experiment 1 50 versus 50 data
#filter_file = "B0708" #post revision experiment 2 50 versus 50 data

#MBP_GFP_mixed = nomixed.loc[nomixed['type']=='MBP+GFP10D_20_80']
#MBP_GFP_mixed = nomixed.loc[nomixed['type']=='MBP+GFP10D_40_60']
#MBP_GFP_mixed = nomixed.loc[nomixed['type']=='MBP+GFP10D_50_50']
MBP_GFP_mixed = nomixed.loc[nomixed['type']=='MBP+GFP10D_60_40']
#MBP_GFP_mixed = nomixed.loc[nomixed['type']=='MBP+GFP10D_80_20']

#MBP_GFP_mixed = MBP_GFP_mixed[MBP_GFP_mixed['file'].str.contains(filter_file) != True]
#print(f"Size of mixture dataframe we are predicting on: {len(MBP_GFP_mixed)}")
#############################################################################
nomixed=eventsDf
prior_NA = len(nomixed)

nomixed=nomixed.loc[nomixed['duration']>300e-6].loc[nomixed['duration']<20e-3] #minimum of 125 samples and maximum of 5000 samples
#nomixed=nomixed.loc[nomixed['duration']>900e-6]

post_NA = len(nomixed)
removed_NA_df = nomixed.dropna()

print(f"Size of dataframe prior to filtering by duration: {prior_NA}")
print(f"Size of dataframe after filtering by duration: {post_NA}")
print(f"Size of dataframe after  filtering by duration and dropping NaNs from dataframe: {len(removed_NA_df)}")

#"MBP Nterm"
#"MBP10D"
#"MBPMBP10D"
#"GFP10D"

updated_df_list = []
type_list = ["MBP10D", "GFP10D"]
#type_list = ["MBP10D", "MBPMBP10D"]
#type_list = ["MBP10D", "MBP Nterm"]
#type_list = ["MBP Nterm", "MBPMBP10D"]
#type_list = ["MBP10D", "MBP Nterm", "MBPMBP10D"]

#Labels = ["MBPD10", "D10MBP", "diMBPD10"]
Labels = ["MBPD10", "GFPD10"]

for label in type_list:
    df = nomixed.loc[nomixed["type"] == label]
    updated_df_list.append(df)
noMixed = pd.concat(updated_df_list)
noMixed['type'].value_counts()

limiting_sample_size = min(noMixed['type'].value_counts())
##limiting_sample_size = 750

print(f"Sample size of each class for training and testing: {limiting_sample_size}")
seedVal_sample = random.randint(0, 200)

#fig=plt.figure(1,clear=True,figsize=(12,80),dpi=300)
#fig, ax = plt.subplots(3, 3, sharex=True, sharey=True)

fig = plt.figure(10,figsize=(15,15),dpi=450)

classification_accuracy = []
confusion_percents = [ [[], []], [[], []] ] #two-way matrix
#confusion_percents = [ [[], [], []], [[], [], []], [[], [], []] ] #three-way matrix

for idx, i in enumerate(range(9)):
    updated_df_list = []
    for label in type_list:
        df = noMixed.loc[noMixed["type"]==label].sample(limiting_sample_size, random_state=seedVal_sample)
        updated_df_list.append(df)
    noMixed = pd.concat(updated_df_list)
    #########################################################
    #MBP_model_data_indices = noMixed.loc[noMixed['type']=='MBP10D'].index.tolist()
    #MBP_GFP_mixed = nomixed.loc[nomixed['type']=='MBP10D'].drop(MBP_model_data_indices)
    #print(f"Size of mixture dataframe we are predicting on: {len(MBP_GFP_mixed)}")
    
    #GFP_model_data_indices = noMixed.loc[noMixed['type']=='GFP10D'].index.tolist()
    #MBP_GFP_mixed = nomixed.loc[nomixed['type']=='GFP10D'].drop(GFP_model_data_indices)
    #print(f"Size of mixture dataframe we are predicting on: {len(MBP_GFP_mixed)}")
    #########################################################
    #noMixed['type'].value_counts()
    x=noMixed.drop(columns=['event','file','I_0','Imax', 'mean', 'std', 'event_start', 'event_end'])#, 'fc_0'])
    x=x.drop(columns=['type','Blockage','mean/I_0', 'duration', 'std/I_0', 'Imin', 'max/std', 'min/duration', 'duration/std'])#'Imax/I_0','min/max','Imin/I_0','min/mean','min/std', 'mean/std'])
    x=x.drop(columns=['min/max', 'min/mean', 'min/std', 'mean/std', 'Imin/I_0', 'Imax/I_0'])#'Imax/I_0','min/max','Imin/I_0','min/mean','min/std', 'mean/std'])
    
#     feature_space = []
#     #remove_fc0 = "_fc0"
#     remove_fc0 = "FC"
#     for (columnName, columnData) in x.iteritems():
#         if remove_fc0 in columnName:
#             pass
#         else:
#             feature_space.append(columnName)
    x = x.loc[:, feature_space]
    y=noMixed['type']
    seedVal_trte_split = random.randint(0, 200)
    x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.2, random_state=seedVal_trte_split) #2 #40 (123 seed was good for all four analytes)
    scaler = sk.preprocessing.StandardScaler()
    x_train = scaler.fit_transform( x_train )
    x_test = scaler.transform( x_test )
    
    print(f"Expected: {y_test.value_counts()}")
    
    
    gbc = GradientBoostingClassifier(n_estimators=200,max_depth=5)
    #rfc = RandomForestClassifier(n_estimators=200, max_depth=5)#, max_features=None)
    #logisticRegr = LogisticRegression(solver = 'lbfgs')
    #knn = KNeighborsClassifier(n_neighbors=5)
    #clf = SVC(decision_function_shape='ovr',probability=True,kernel='rbf', 
               #C=200,verbose=True,gamma='scale', class_weight = None)

    gbc.fit(x_train, y_train)
    #clf.fit(x_train, y_train)

    y_pred=gbc.predict(x_test)
    unique, counts = np.unique(y_pred, return_counts=True)
    print(f"Predicted: {dict(zip(unique, counts))}")
    classification_accuracy.append(accuracy_score(y_test,y_pred)*100)
    
    #conf_mat = confusion_matrix(y_true=y_test, y_pred=y_pred)#, normalize='true' )
    conf_mat = confusion_matrix(y_true=y_test, y_pred=y_pred,labels=type_list,normalize="true")
    conf_mat_raw = confusion_matrix(y_true=y_test, y_pred=y_pred,labels=type_list,normalize=None)
    print('Confusion matrix:\n', conf_mat)
    ax = fig.add_subplot(330+1+i)
    ax.grid(False)
    cax = ax.matshow(conf_mat, cmap=plt.cm.Blues,vmin=0,vmax=1)
    for k in range(conf_mat.shape[0]):
        for j in range(conf_mat.shape[1]):
            confusion_percents[k][j].append(conf_mat[k, j]*100)
            if (k == 0 and j == 0) or (k == 1 and j == 1) or (k == 2 and j == 2):
                ax.text(x=j, y=k,s=f"{conf_mat[k, j]*100:0.1f} %\n\n{conf_mat_raw[k, j]}", va='center', ha='center', size='x-large', color='white')
            else:
                ax.text(x=j, y=k,s=f"{conf_mat[k, j]*100:0.1f} %\n\n{conf_mat_raw[k, j]}", va='center', ha='center', size='x-large', color='k')

    ax.set_title(f"Shuffle {i+1}, {accuracy_score(y_test,y_pred)*100:.1f} %")
    #plt.subplot(3,3,idx+1)
    
    #labels = type_list
    labels = Labels
    ax.xaxis.tick_bottom()
    ax.set_xticklabels([''] + labels)
    ax.set_yticklabels([''] + labels,rotation="vertical",verticalalignment='center')
    plt.xlabel("Predicted")
    plt.ylabel("Expected")
    
#fig.text(0.5, 0.04, "Predicted", ha='center', fontsize="xx-large", weight='bold')
#fig.text(0.04, 0.5, "Expected", va='center', rotation='vertical', fontsize="xx-large", weight='bold')
#fig.tight_layout()
#fig.subplots_adjust(top=.75)

#plt.savefig('All_Confusions_MBPD10_vs_GFPD10.png', transparent=True)
#plt.savefig('All_Confusions_MBPD10_vs_GFPD10.svg', transparent=True)

#plt.savefig('All_Confusions_MBPD10_vs_D10MBP_vs_diMBPD10.png', transparent=True)
#plt.savefig('All_Confusions_MBPD10_vs_D10MBP_vs_diMBPD10.svg', transparent=True)
  



    mixed=MBP_GFP_mixed.drop(columns=['event','file','I_0','Imax', 'mean', 'std', 'event_start', 'event_end'])#, 'fc_0'])

    #More features are removed
    mixed = mixed.loc[:, feature_space]

    mixed_scaled = scaler.transform(mixed)
    mixed_exp_calls = gbc.predict(mixed_scaled)
    #mixed_exp_calls = gbc.predict(mixed)
    #mixed_exp_calls = clf.predict(mixed_scaled)
    unique, counts = np.unique(mixed_exp_calls, return_counts=True)
    print(dict(zip(unique, counts)))
    
    for call, count in zip(unique, counts):
        calls_dict[call].append(count)

print('Mean classification accuracy of GBC model: {}'.format(np.mean(classification_accuracy)))  
###############################################################################################################################################
confusion_percents_array = np.array(confusion_percents)
print(confusion_percents_array)

plusminus = u"\u00B1"
fig = plt.figure(11,figsize=(6,6),dpi=450)
ax = fig.add_subplot(1,1,1)
ax.grid(False)
cax = ax.matshow(conf_mat, cmap=plt.cm.Blues,vmin=0,vmax=1)
for k in range(confusion_percents_array.shape[0]):
    for j in range(confusion_percents_array.shape[1]):
        if (k == 0 and j == 0) or (k == 1 and j == 1) or (k == 2 and j == 2):
            ax.text(x=j, y=k,s=f"{np.mean(confusion_percents_array[k, j]):0.1f}{plusminus}{np.std(confusion_percents_array[k, j]):0.2f}%", va='center', ha='center', size='x-large', color='white')
        else:
            ax.text(x=j, y=k,s=f"{np.mean(confusion_percents_array[k, j]):0.1f}{plusminus}{np.std(confusion_percents_array[k, j]):0.2f}%", va='center', ha='center', size='x-large', color='k')
labels = Labels
ax.set_title(f"Mean Classification Accuracy: {np.mean(classification_accuracy):.1f} %")
ax.xaxis.tick_bottom()
ax.set_xticklabels([''] + labels)
ax.set_yticklabels([''] + labels,rotation="vertical",verticalalignment='center')
plt.xlabel("Predicted")
plt.ylabel("Expected")

#plt.savefig('Mean_Confusion_MBPD10_vs_GFPD10.png', transparent=True)
#plt.savefig('Mean_Confusion_MBPD10_vs_GFPD10.svg', transparent=True)

#plt.savefig('Mean_Confusion_MBPD10_vs_D10MBP_vs_diMBPD10.png', transparent=True)
#plt.savefig('Mean_Confusion_MBPD10_vs_D10MBP_vs_diMBPD10.svg', transparent=True)
###############################################################################################################################################



#     if i==0:
#         single_fig=plt.figure(11,figsize=(3,3.5),dpi=450)
#         ax=single_fig.gca().matshow(conf_mat, cmap=plt.cm.Blues,vmin=0,vmax=1)
#         for k in range(conf_mat.shape[0]):
#             for j in range(conf_mat.shape[1]):
#                 plt.text(x=j, y=k,s=f"{conf_mat[k, j]*100:0.1f} %\n\n{conf_mat_raw[k, j]}", va='center', ha='center', size='x-large')
# #                 plt.text(x=j, y=k,s=f"{conf_mat_raw[k, j]}", va='bottom', ha='center', size='x-large')
#         plt.title(f"GBC test confusion matrix\naccuracy: {accuracy_score(y_test,y_pred)*100:.1f} %")
# #         accuracies.append(accuracy_score(y_test,y_pred)*100)
#         plt.gca().xaxis.tick_bottom()
#         plt.xlabel("Predicted")
#         plt.ylabel("Expected")
        
#         plt.gca().set_xticklabels([''] + labels)
#         plt.gca().set_yticklabels([''] + labels,rotation="vertical",verticalalignment='center')
        
    
    
    
        
        
        
        #plt.figure(10)
        
#fig.colorbar(cax)

#fig.ylabel('Expected')
#fig.xlabel('Predicted')
#     plt.show()
#fig.title('Mean classification accuracy of GBC model: {}'.format(np.mean(classification_accuracy)))

#     title = f"Normalized confusion matrix iteration: {idx+1}"
#     #class_names = ['GFP', 'MBP']
#     disp = plot_confusion_matrix(gbc, x_test, y_test,
#                                  display_labels=type_list,#class_names,
#                                  cmap=plt.cm.binary,
#                                  #cmap=plt.cm.RdYlGn,
#                                  normalize='true')
#     disp.ax_.set_title(title)




#plt.savefig('MBPD10_vs_GFPD10_Confusion.png', transparent=True)
#plt.savefig('MBPD10_vs_GFPD10_Confusion.svg', transparent=True)

#plt.savefig('MBPD10_vs_MBPD10_Dimer_Confusion.png', transparent=True)
#plt.savefig('MBPD10_vs_MBPD10_Dimer_Confusion.svg', transparent=True)

#plt.savefig('MBPD10_vs_MBP Nterm_Confusion.png', transparent=True)
#plt.savefig('MBPD10_vs_MBP Nterm_Confusion.svg', transparent=True)

#plt.savefig('Dimer_vs_MBP Nterm_Confusion.png', transparent=True)
#plt.savefig('Dimer_vs_MBP Nterm_Confusion.svg', transparent=True)

#plt.savefig('MBPD10_vs_MBP Nterm_vs_MBPD10_Dimer_Confusion.png', transparent=True)
#plt.savefig('MBPD10_vs_MBP Nterm_vs_MBPD10_Dimer_Confusion.svg', transparent=True)


#save_results_to = 'Figures/Updated_Figures/GFP_WT_150mV_Figs/'
#plt.savefig(save_results_to + 'MBPD10_120mV_Segmentation_4_components.png', transparent=True, dpi = 300)
#plt.savefig(save_results_to + 'MBPD10_120mV_Segmentation_4_components.svg', transparent=True, dpi = 300)



#size_0 = len(test_df.loc[test_df['type']==x1])
#size_100perc_mod_test = len(test_df.loc[test_df['type']==x2])
    
#if size_0perc_mod_test <= size_100perc_mod_test:
#    original_test_100_mod = test_df.loc[test_df['type']==x2].sample(size_0perc_mod_test)
#    original_test_0_mod = test_df.loc[test_df['type']==x1]
#    size_0perc_mod_test = len(original_test_0_mod)
#    size_100perc_mod_test = len(original_test_100_mod)
#else:
#    original_test_0_mod = test_df.loc[test_df['type']==x1].sample(size_100perc_mod_test)
#    original_test_100_mod = test_df.loc[test_df['type']==x2]
#    size_0perc_mod_test = len(original_test_0_mod)
#    size_100perc_mod_test = len(original_test_100_mod)
    
#print(f'Size of original X-test after balancing: 0% mod: {size_0perc_mod_test}\n 100% mod: {size_100perc_mod_test}')



# #Resampling to make MBP10D and GFP10D the same size for training and testing
# #n = len(nomixed.loc[nomixed['type']=='GFP10D'])
# n = len(nomixed.loc[nomixed['type']=='MBP Nterm'])
# #n = len(nomixed.loc[nomixed['type']=='MBPMBP10D'])
# seedVal = random.randint(0, 200)
# #################################################################################
# #resamp_MBP10D = nomixed.loc[nomixed["type"]=="MBP10D"].sample(n, random_state=seedVal)
# #resamp_GFP10D = nomixed.loc[nomixed["type"]=="GFP10D"].sample(n, random_state=seedVal)
# #noMixed = pd.concat([resamp_MBP10D, resamp_GFP10D])
# #################################################################################

# #resamp_MBP10D = nomixed.loc[nomixed["type"]=="MBP10D"].sample(n, random_state=1)
# #resamp_MBPMBP10D = nomixed.loc[nomixed["type"]=="MBPMBP10D"].sample(n, random_state=1)
# #noMixed = pd.concat([resamp_MBP10D, resamp_MBPMBP10D])

# #################################################################################

# #################################################################################

# resamp_MBP10D = nomixed.loc[nomixed["type"]=="MBP10D"].sample(n, random_state=seedVal)
# resamp_NtermMPB10D = nomixed.loc[nomixed["type"]=="MBP Nterm"].sample(n, random_state=seedVal)
# noMixed = pd.concat([resamp_MBP10D, resamp_NtermMPB10D])


# x=noMixed.drop(columns=['event','file','I_0','Imax', 'mean', 'std', 'event_start', 'event_end'])#, 'fc_0'])
# print(f"How many samples in the training and testing combined dataframe: \n{x['type'].value_counts()}")
# print(" ... \n")

# #More features are removed
# x=x.drop(columns=['type','Blockage','mean/I_0', 'duration', 'std/I_0', 'Imin', 'max/std', 'min/duration', 'duration/std'])#'Imax/I_0','min/max','Imin/I_0','min/mean','min/std', 'mean/std'])
# x=x.drop(columns=['min/max', 'min/mean', 'min/std', 'mean/std', 'Imin/I_0', 'Imax/I_0'])#'Imax/I_0','min/max','Imin/I_0','min/mean','min/std', 'mean/std'])


# #training_features = ['min/std','Imax/I_0','sm1','Imin/I_0','min/mean','sm0','ss0','sm1','ss1',
# #                     'sm2','ss2','seg0_fc4','seg1_fc4','seg2_fc4','sm4','ss4','sm5','ss5',
# #                     'sm6','ss6','seg4_fc4','seg5_fc4','seg6_fc4'] #subsequences Mean
# #x = x.loc[:, training_features]

# feature_space = []
# remove_fc0 = "_fc0"
# for (columnName, columnData) in x.iteritems():
#     if remove_fc0 in columnName:
#         pass
#     else:
#         feature_space.append(columnName)

# x = x.loc[:, feature_space]





# ######################
# y=noMixed['type']
# #y=nomixed['type']
# ######################

# features = []
# for (columnName, columnData) in x.iteritems():
#     features.append(columnName)

# print(f"The following list contains the feature space used for training the model: \n{features}")

# seedVal = random.randint(0, 200)
# x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.2, random_state=seedVal) #2 #40 (123 seed was good for all four analytes)
# scaler = sk.preprocessing.StandardScaler()
# x_train = scaler.fit_transform( x_train )
# x_test = scaler.transform( x_test )
# print("...")



# gbc = GradientBoostingClassifier(n_estimators=200,max_depth=5)
# #rfc = RandomForestClassifier(n_estimators=200, max_depth=5)#, max_features=None)
# #logisticRegr = LogisticRegression(solver = 'lbfgs')
# #knn = KNeighborsClassifier(n_neighbors=5)
# #clf = SVC(decision_function_shape='ovr',probability=True,kernel='rbf', 
# #          C=200,verbose=True,gamma='scale', class_weight = None)

# gbc.fit(x_train, y_train)
# #clf.fit(x_train, y_train)

# y_pred=gbc.predict(x_test)
# #y_pred=clf.predict(x_test)


# print('...')
# print('Size of training set {}.'.format(len(x_train)))
# print('Size of testing set {}.'.format(len(x_test)))
# print('Classification accuracy of GBC model: {} and the training test split seed value: {}'.format(accuracy_score(y_test,y_pred), seedVal))
# #print('Classification accuracy of SVM model: {}'.format(accuracy_score(y_test,y_pred)))
# #print(classification_report(y_test, y_pred))


print(calls_dict)


In [ ]:
###Save and plot calls from ratios###
n_20_80 = len(nomixed.loc[nomixed['type']=='MBP+GFP10D_20_80'])
n_40_60 = len(nomixed.loc[nomixed['type']=='MBP+GFP10D_40_60'])
n_50_50 = len(nomixed.loc[nomixed['type']=='MBP+GFP10D_50_50'])
n_60_40 = len(nomixed.loc[nomixed['type']=='MBP+GFP10D_60_40'])
n_80_20 = len(nomixed.loc[nomixed['type']=='MBP+GFP10D_80_20'])    

#calls_dict_80_20 = calls_dict
#print(calls_dict_80_20)

#calls_dict_60_40 = calls_dict
#print(calls_dict_60_40)

#calls_dict_40_60 = calls_dict
#print(calls_dict_40_60)

#calls_dict_20_80 = calls_dict
#print(calls_dict_20_80)

#calls_dict_50_50 = calls_dict
#print(calls_dict_50_50)

#calls_dict_100_0 = calls_dict
#print(calls_dict_100_0)

calls_dict_0_100 = calls_dict
print(calls_dict_0_100)

# GFP_mean_20_80 = np.mean(np.array(calls_dict_20_80['GFP10D'])/n_20_80)
# GFP_std_20_80 = np.std(np.array(calls_dict_20_80['GFP10D'])/n_20_80)
# print(GFP_mean_20_80)
# print(GFP_std_20_80)
# GFP_mean_40_60 = np.mean(np.array(calls_dict_40_60['GFP10D'])/n_40_60)
# GFP_std_40_60 = np.std(np.array(calls_dict_40_60['GFP10D'])/n_40_60)
# print(GFP_mean_40_60)
# print(GFP_std_40_60)
# GFP_mean_50_50 = np.mean(np.array(calls_dict_50_50['GFP10D'])/n_50_50)
# GFP_std_50_50 = np.std(np.array(calls_dict_50_50['GFP10D'])/n_50_50)
# print(GFP_mean_50_50)
# print(GFP_std_50_50)
# GFP_mean_60_40 = np.mean(np.array(calls_dict_60_40['GFP10D'])/n_60_40)
# GFP_std_60_40 = np.std(np.array(calls_dict_60_40['GFP10D'])/n_60_40)
# print(GFP_mean_60_40)
# print(GFP_std_60_40)
# print("...")
# GFP_mean_80_20 = np.mean(np.array(calls_dict_80_20['GFP10D'])/n_80_20)
# GFP_std_80_20 = np.std(np.array(calls_dict_80_20['GFP10D'])/n_80_20)
# print(GFP_mean_80_20)
# print(GFP_std_80_20)


# MBP_mean_20_80 = np.mean(np.array(calls_dict_20_80['MBP10D'])/n_20_80)
# MBP_std_20_80 = np.std(np.array(calls_dict_20_80['MBP10D'])/n_20_80)
# print(MBP_mean_20_80)
# print(MBP_std_20_80)
# MBP_mean_40_60 = np.mean(np.array(calls_dict_40_60['MBP10D'])/n_40_60)
# MBP_std_40_60 = np.std(np.array(calls_dict_40_60['MBP10D'])/n_40_60)
# print(MBP_mean_40_60)
# print(MBP_std_40_60)
# MBP_mean_50_50 = np.mean(np.array(calls_dict_50_50['MBP10D'])/n_50_50)
# MBP_std_50_50 = np.std(np.array(calls_dict_50_50['MBP10D'])/n_50_50)
# print(MBP_mean_50_50)
# print(MBP_std_50_50)
# MBP_mean_60_40 = np.mean(np.array(calls_dict_60_40['MBP10D'])/n_60_40)
# MBP_std_60_40 = np.std(np.array(calls_dict_60_40['MBP10D'])/n_60_40)
# print(MBP_mean_60_40)
# print(MBP_std_60_40)
# MBP_mean_80_20 = np.mean(np.array(calls_dict_80_20['MBP10D'])/n_80_20)
# MBP_std_80_20 = np.std(np.array(calls_dict_80_20['MBP10D'])/n_80_20)
# print(MBP_mean_80_20)
# print(MBP_std_80_20)

In [ ]:
###Save prediction results for each ratio###

import pickle

# with open(f"Model_Calls/80_20_MBP_GFP_results", "wb") as fp:   #Pickling
#     pickle.dump(calls_dict_80_20, fp)
    
# with open(f"Model_Calls/60_40_MBP_GFP_results", "wb") as fp:   #Pickling
#     pickle.dump(calls_dict_60_40, fp)
    
# with open(f"Model_Calls/40_60_MBP_GFP_results", "wb") as fp:   #Pickling
#     pickle.dump(calls_dict_40_60, fp)
    
# with open(f"Model_Calls/20_80_MBP_GFP_results", "wb") as fp:   #Pickling
#     pickle.dump(calls_dict_20_80, fp)

#with open(f"Model_Calls/100_0_MBP_GFP_results", "wb") as fp:   #Pickling
#    pickle.dump(calls_dict_100_0, fp)
    
# with open(f"Model_Calls/0_100_MBP_GFP_results", "wb") as fp:   #Pickling
#     pickle.dump(calls_dict_0_100, fp)
    
    
# with open(f"Model_Calls/50_50_MBP_GFP_results_pre_revisions", "wb") as fp:   #Pickling
#     pickle.dump(calls_dict_50_50, fp)
    
# with open(f"Model_Calls/50_50_MBP_GFP_results_post_revisions_1", "wb") as fp:   #Pickling
#     pickle.dump(calls_dict_50_50, fp)

# with open(f"Model_Calls/50_50_MBP_GFP_results_post_revisions_2", "wb") as fp:   #Pickling
#     pickle.dump(calls_dict_50_50, fp)
    
# with open(f"Model_Calls/50_50_MBP_GFP_results_post_revisions_combined", "wb") as fp:   #Pickling
#     pickle.dump(calls_dict_50_50, fp)
    
# with open(f"Model_Calls/50_50_MBP_GFP_results_all_combined", "wb") as fp:   #Pickling
#     pickle.dump(calls_dict_50_50, fp)

In [ ]:
###Load prediction results for each ratio###

import pickle

with open(f"Model_Calls/80_20_MBP_GFP_results", "rb") as fp:   #Pickling
    calls_dict_80_20 = pickle.load(fp)
print(calls_dict_80_20)
    
with open(f"Model_Calls/60_40_MBP_GFP_results", "rb") as fp:   #Pickling
    calls_dict_60_40 = pickle.load(fp)
print(calls_dict_60_40)

with open(f"Model_Calls/40_60_MBP_GFP_results", "rb") as fp:   #Pickling
    calls_dict_40_60 = pickle.load(fp)
print(calls_dict_40_60)
    
with open(f"Model_Calls/20_80_MBP_GFP_results", "rb") as fp:   #Pickling
    calls_dict_20_80 = pickle.load(fp)
print(calls_dict_20_80)
    
with open(f"Model_Calls/50_50_MBP_GFP_results_all_combined", "rb") as fp:   #Pickling
    calls_dict_50_50 = pickle.load(fp)
print(calls_dict_50_50) 
    
with open(f"Model_Calls/100_0_MBP_GFP_results", "rb") as fp:   #Pickling
    calls_dict_100_0 = pickle.load(fp)
print(calls_dict_100_0) 
    
with open(f"Model_Calls/0_100_MBP_GFP_results", "rb") as fp:   #Pickling
    calls_dict_0_100 = pickle.load(fp)
print(calls_dict_0_100)
        
   
with open(f"Model_Calls/50_50_MBP_GFP_results_pre_revisions", "rb") as fp:   #Pickling
    calls_dict_50_50_pre = pickle.load(fp)
print(calls_dict_50_50_pre)

with open(f"Model_Calls/50_50_MBP_GFP_results_post_revisions_1", "rb") as fp:   #Pickling
    calls_dict_50_50_post1 = pickle.load(fp)
with open(f"Model_Calls/50_50_MBP_GFP_results_post_revisions_1", "rb") as fp:   #Pickling
    calls_dict_50_50_post2 = pickle.load(fp)
    
with open(f"Model_Calls/50_50_MBP_GFP_results_post_revisions_combined", "rb") as fp:   #Pickling
    calls_dict_50_50_postcombined = pickle.load(fp)
print(calls_dict_50_50_postcombined)
    
    
n_20_80 = len(nomixed.loc[nomixed['type']=='MBP+GFP10D_20_80'])
n_40_60 = len(nomixed.loc[nomixed['type']=='MBP+GFP10D_40_60'])

n_60_40 = len(nomixed.loc[nomixed['type']=='MBP+GFP10D_60_40'])
n_80_20 = len(nomixed.loc[nomixed['type']=='MBP+GFP10D_80_20']) 

n_50_50 = len(nomixed.loc[nomixed['type']=='MBP+GFP10D_50_50'])

n_0_100 = calls_dict_0_100['MBP10D'][0] + calls_dict_0_100['GFP10D'][0]
n_100_0 = calls_dict_100_0['MBP10D'][0] + calls_dict_100_0['GFP10D'][0]

n_50_50_pre = calls_dict_50_50_pre['MBP10D'][0] + calls_dict_50_50_pre['GFP10D'][0]
n_50_50_post1 = calls_dict_50_50_post1['MBP10D'][0] + calls_dict_50_50_post1['GFP10D'][0]  
n_50_50_post2 = calls_dict_50_50_post2['MBP10D'][0] + calls_dict_50_50_post2['GFP10D'][0] 
n_50_50_postcombined = calls_dict_50_50_postcombined['MBP10D'][0] + calls_dict_50_50_postcombined['GFP10D'][0]  
    
MBP_mean_20_80 = np.mean(np.array(calls_dict_20_80['MBP10D'])/n_20_80)
MBP_std_20_80 = np.std(np.array(calls_dict_20_80['MBP10D'])/n_20_80)

MBP_mean_40_60 = np.mean(np.array(calls_dict_40_60['MBP10D'])/n_40_60)
MBP_std_40_60 = np.std(np.array(calls_dict_40_60['MBP10D'])/n_40_60)

MBP_mean_60_40 = np.mean(np.array(calls_dict_60_40['MBP10D'])/n_60_40)
MBP_std_60_40 = np.std(np.array(calls_dict_60_40['MBP10D'])/n_60_40)

MBP_mean_80_20 = np.mean(np.array(calls_dict_80_20['MBP10D'])/n_80_20)
MBP_std_80_20 = np.std(np.array(calls_dict_80_20['MBP10D'])/n_80_20)

MBP_mean_50_50 = np.mean(np.array(calls_dict_50_50['MBP10D'])/n_50_50)
MBP_std_50_50 = np.std(np.array(calls_dict_50_50['MBP10D'])/n_50_50)

MBP_mean_0_100 = np.mean(np.array(calls_dict_0_100['MBP10D'])/n_0_100)
MBP_std_0_100 = np.std(np.array(calls_dict_0_100['MBP10D'])/n_0_100)

MBP_mean_100_0 = np.mean(np.array(calls_dict_100_0['MBP10D'])/n_100_0)
MBP_std_100_0 = np.std(np.array(calls_dict_100_0['MBP10D'])/n_100_0)
#############################################################################################
MBP_mean_50_50_pre = np.mean(np.array(calls_dict_50_50_pre['MBP10D'])/n_50_50_pre)
MBP_std_50_50_pre = np.std(np.array(calls_dict_50_50_pre['MBP10D'])/n_50_50_pre)

MBP_mean_50_50_post1 = np.mean(np.array(calls_dict_50_50_post1['MBP10D'])/n_50_50_post1)
MBP_std_50_50_post1 = np.std(np.array(calls_dict_50_50_post1['MBP10D'])/n_50_50_post1)

MBP_mean_50_50_post2 = np.mean(np.array(calls_dict_50_50_post2['MBP10D'])/n_50_50_post2)
MBP_std_50_50_post2 = np.std(np.array(calls_dict_50_50_post2['MBP10D'])/n_50_50_post2)

MBP_mean_50_50_postcombined = np.mean(np.array(calls_dict_50_50_postcombined['MBP10D'])/n_50_50_postcombined)
MBP_std_50_50_postcombined = np.std(np.array(calls_dict_50_50_postcombined['MBP10D'])/n_50_50_postcombined)



GFP_mean_20_80 = np.mean(np.array(calls_dict_20_80['GFP10D'])/n_20_80)
GFP_std_20_80 = np.std(np.array(calls_dict_20_80['GFP10D'])/n_20_80)

GFP_mean_40_60 = np.mean(np.array(calls_dict_40_60['GFP10D'])/n_40_60)
GFP_std_40_60 = np.std(np.array(calls_dict_40_60['GFP10D'])/n_40_60)

GFP_mean_60_40 = np.mean(np.array(calls_dict_60_40['GFP10D'])/n_60_40)
GFP_std_60_40 = np.std(np.array(calls_dict_60_40['GFP10D'])/n_60_40)

GFP_mean_80_20 = np.mean(np.array(calls_dict_80_20['GFP10D'])/n_80_20)
GFP_std_80_20 = np.std(np.array(calls_dict_80_20['GFP10D'])/n_80_20)

GFP_mean_50_50 = np.mean(np.array(calls_dict_50_50['GFP10D'])/n_50_50)
GFP_std_50_50 = np.std(np.array(calls_dict_50_50['GFP10D'])/n_50_50)
#############################################################################################
GFP_mean_50_50_pre = np.mean(np.array(calls_dict_50_50_pre['GFP10D'])/n_50_50_pre)
GFP_std_50_50_pre = np.std(np.array(calls_dict_50_50_pre['GFP10D'])/n_50_50_pre)

GFP_mean_50_50_post1 = np.mean(np.array(calls_dict_50_50_post1['GFP10D'])/n_50_50_post1)
GFP_std_50_50_post1 = np.std(np.array(calls_dict_50_50_post1['GFP10D'])/n_50_50_post1)

GFP_mean_50_50_post2 = np.mean(np.array(calls_dict_50_50_post2['GFP10D'])/n_50_50_post2)
GFP_std_50_50_post2 = np.std(np.array(calls_dict_50_50_post2['GFP10D'])/n_50_50_post2)

GFP_mean_50_50_postcombined = np.mean(np.array(calls_dict_50_50_postcombined['GFP10D'])/n_50_50_postcombined)
GFP_std_50_50_postcombined = np.std(np.array(calls_dict_50_50_postcombined['GFP10D'])/n_50_50_postcombined)

GFP_mean_0_100 = np.mean(np.array(calls_dict_0_100['GFP10D'])/n_0_100)
GFP_std_0_100 = np.std(np.array(calls_dict_0_100['GFP10D'])/n_0_100)

GFP_mean_100_0 = np.mean(np.array(calls_dict_100_0['GFP10D'])/n_100_0)
GFP_std_100_0 = np.std(np.array(calls_dict_100_0['GFP10D'])/n_100_0)

In [ ]:
###Plotting MBP-GFP ratio calls###


####Updated Figure 3 that has all 4 genes in one plot####

from sklearn.metrics import RocCurveDisplay
import seaborn as sns
from matplotlib import gridspec
import matplotlib.patches as mpatches
import matplotlib.font_manager
from matplotlib.ticker import AutoMinorLocator
from matplotlib.ticker import MultipleLocator
import pickle

sns.set(style='white')
    

# n_20_80 = len(nomixed.loc[nomixed['type']=='MBP+GFP10D_20_80'])
# n_40_60 = len(nomixed.loc[nomixed['type']=='MBP+GFP10D_40_60'])
# n_50_50 = len(nomixed.loc[nomixed['type']=='MBP+GFP10D_50_50'])
# n_60_40 = len(nomixed.loc[nomixed['type']=='MBP+GFP10D_60_40'])
# n_80_20 = len(nomixed.loc[nomixed['type']=='MBP+GFP10D_80_20']) 

fig, ax = plt.subplots(figsize=(8,8),dpi=300)

#x_ticks = range(0,6,1)
x_ticks = np.arange(0,11,1)
ax.set_xticks(x_ticks)

ax.set_xticklabels(['0%','10%', '20%', '30%', '40%', '50%', '60%', '70%', '80%', '90%', '100%'],
                    fontdict = {'fontsize' : 15})


# ax.set_xticklabels([f'0-100 MBP:GFP\n(n={n_0_100})\n{round(MBP_mean_0_100, 2)}:{round(GFP_mean_0_100, 2)}',
#                     f'20-80 MBP:GFP\n(n={n_20_80})\n{round(MBP_mean_20_80, 2)}:{round(GFP_mean_20_80, 2)}',
#                     f'30-70 MBP:GFP',
#                     f'40-60 MBP:GFP\n(n={n_40_60})\n{round(MBP_mean_40_60, 2)}:{round(GFP_mean_40_60, 2)}', 
#                     f'50-50 MBP:GFP\n(n={n_50_50})\n{round(MBP_mean_50_50, 2)}:{round(GFP_mean_50_50, 2)}', 
#                     f'60-40 MBP:GFP\n(n={n_60_40})\n{round(MBP_mean_60_40, 2)}:{round(GFP_mean_60_40, 2)}',
#                     f'70-30 GFP:MBP',
#                     f'80-20 MBP:GFP\n(n={n_80_20})\n{round(MBP_mean_80_20, 2)}:{round(GFP_mean_80_20, 2)}',
#                     f'100-0 MBP:GFP\n(n={n_100_0})\n{round(MBP_mean_100_0, 2)}:{round(GFP_mean_100_0, 2)}'],
#                     fontdict = {'fontsize' : 7})#, fontdict

# ax.set_xticklabels([f'PSMB2\n({PSMB2_KMER})', f'MCM5\n({MCM5_KMER})', f'PRPSAP1\n({PRPSAP1_KMER})', 
#                     f'MRPS14\n({MRPS14_KMER})', f'PTTG1IP\n({PTTG1IP_KMER})', f'RNF7\n({RNF7_KMER})'], fontdict = {'fontsize' : 16})#, fontdict = {'fontsize' : 7})
ax.set_ylim([0, 1])
ax.set_xlim([-0.5, 10.5])



#ax.set_ylabel(f'$\psi$ called by model (%)', fontdict = {'fontsize' : 22})
ax.set_ylabel(f'Predicted Ratio (GBC Classification)', fontdict = {'fontsize' : 22})
ax.set_xlabel(f'Actual Ratio (%)', fontdict = {'fontsize' : 22})

#y_ticks = np.arange(0.,1.1,0.1)
y_ticks = np.arange(0.,1.1,0.2)
ax.set_yticks(y_ticks)

# ax.set_yticklabels(["0", "0.1", "0.2", "0.3", 
#           "0.4", "0.5", "0.6", "0.7", 
#           "0.8", "0.9", "1"], fontdict = {'fontsize' : 20})

ax.set_yticklabels(["0%", "20%", "40%", "60%", "80%", "100%"], fontdict = {'fontsize' : 15})


# ax.set_yticklabels(["0", "10", "20", "30", 
#           "40", "50", "60", "70", 
#           "80", "90", "100"], fontdict = {'fontsize' : 22})


ax.yaxis.set_ticks_position('both')
ax.xaxis.set_ticks_position('both')
#ax1.tick_params(axis="y",direction="in", pad=5)
#ax1.tick_params(axis="x",direction="in", pad=5)
  
    
ax.tick_params(axis="y",direction="in", pad=5)
ax.tick_params(axis="x",direction="in", pad=5)
minor_locator_x = AutoMinorLocator(2)
minor_locator_y = AutoMinorLocator(2)
ax.tick_params(which='minor', length=2.5, width=0.5, direction='in')
ax.xaxis.set_minor_locator(minor_locator_x)
ax.yaxis.set_minor_locator(minor_locator_y)
ax.grid(False)

violin_parts_0_100_MBP = plt.errorbar(0, MBP_mean_0_100, yerr=MBP_std_0_100, fmt='o', solid_capstyle='projecting', 
                                   capsize=7, color='red', linestyle=None, lw=1, markersize=7, elinewidth=1, label=f"MBP")  
violin_parts_0_100_GFP = plt.errorbar(0, GFP_mean_0_100, yerr=GFP_std_0_100, fmt='o', solid_capstyle='projecting', 
                               capsize=7, color='green', linestyle=None, lw=1, markersize=7, elinewidth=1, label=f"GFP")

violin_parts_20_80_MBP = plt.errorbar(2, MBP_mean_20_80, yerr=MBP_std_20_80, fmt='o', solid_capstyle='projecting', 
                                   capsize=7, color='red', linestyle=None, lw=1, markersize=7, elinewidth=1, label=f"MBP")  
violin_parts_20_80_GFP = plt.errorbar(2, GFP_mean_20_80, yerr=GFP_std_20_80, fmt='o', solid_capstyle='projecting', 
                               capsize=7, color='green', linestyle=None, lw=1, markersize=7, elinewidth=1, label=f"GFP")

violin_parts_40_60_MBP = plt.errorbar(4, MBP_mean_40_60, yerr=MBP_std_40_60, fmt='o', solid_capstyle='projecting', 
                                   capsize=7, color='red', linestyle=None, lw=1, markersize=7, elinewidth=1, label=f"MBP")  
violin_parts_40_60_GFP = plt.errorbar(4, GFP_mean_40_60, yerr=GFP_std_40_60, fmt='o', solid_capstyle='projecting', 
                               capsize=7, color='green', linestyle=None, lw=1, markersize=7, elinewidth=1, label=f"GFP")

violin_parts_50_50_MBP = plt.errorbar(5, MBP_mean_50_50, yerr=MBP_std_50_50, fmt='o', solid_capstyle='projecting', 
                                   capsize=7, color='red', linestyle=None, lw=1, markersize=7, elinewidth=1, label=f"MBP")  
violin_parts_50_50_GFP = plt.errorbar(5, GFP_mean_50_50, yerr=GFP_std_50_50, fmt='o', solid_capstyle='projecting', 
                               capsize=7, color='green', linestyle=None, lw=1, markersize=7, elinewidth=1, label=f"GFP")

violin_parts_60_40_MBP = plt.errorbar(6, MBP_mean_60_40, yerr=MBP_std_60_40, fmt='o', solid_capstyle='projecting', 
                                   capsize=7, color='red', linestyle=None, lw=1, markersize=7, elinewidth=1, label=f"MBP")  
violin_parts_60_40_GFP = plt.errorbar(6, GFP_mean_60_40, yerr=GFP_std_60_40, fmt='o', solid_capstyle='projecting', 
                               capsize=7, color='green', linestyle=None, lw=1, markersize=7, elinewidth=1, label=f"GFP")

violin_parts_80_20_MBP = plt.errorbar(8, MBP_mean_80_20, yerr=MBP_std_80_20, fmt='o', solid_capstyle='projecting', 
                                   capsize=7, color='red', linestyle=None, lw=1, markersize=7, elinewidth=1, label=f"MBP")  
violin_parts_80_20_GFP = plt.errorbar(8, GFP_mean_80_20, yerr=GFP_std_80_20, fmt='o', solid_capstyle='projecting', 
                               capsize=7, color='green', linestyle=None, lw=1, markersize=7, elinewidth=1, label=f"GFP")

violin_parts_100_0_MBP = plt.errorbar(10, MBP_mean_100_0, yerr=MBP_std_100_0, fmt='o', solid_capstyle='projecting', 
                                   capsize=7, color='red', linestyle=None, lw=1, markersize=7, elinewidth=1, label=f"MBP")  
violin_parts_100_0_GFP = plt.errorbar(10, GFP_mean_100_0, yerr=GFP_std_100_0, fmt='o', solid_capstyle='projecting', 
                               capsize=7, color='green', linestyle=None, lw=1, markersize=7, elinewidth=1, label=f"GFP")
       
    

    

violin_parts_50_50_MBP_pre = plt.errorbar(5, MBP_mean_50_50_pre, yerr=MBP_std_50_50_pre, fmt='X', solid_capstyle='projecting', 
                                   capsize=7, color='red', linestyle=None, lw=1, markersize=7, elinewidth=1, label=f"MBP", alpha=0.33)  
violin_parts_50_50_GFP_pre = plt.errorbar(5, GFP_mean_50_50_pre, yerr=GFP_std_50_50_pre, fmt='X', solid_capstyle='projecting', 
                                   capsize=7, color='green', linestyle=None, lw=1, markersize=7, elinewidth=1, label=f"GFP", alpha=0.33)

# violin_parts_50_50_MBP_post1 = plt.errorbar(3-0.25, MBP_mean_50_50_post1, yerr=MBP_std_50_50_post1, fmt='s', solid_capstyle='projecting', 
#                                    capsize=7, color='red', linestyle=None, lw=1, markersize=7, elinewidth=1, label=f"MBP")  
# violin_parts_50_50_GFP_post1 = plt.errorbar(3+0.25, GFP_mean_50_50_post1, yerr=GFP_std_50_50_post1, fmt='s', solid_capstyle='projecting', 
#                                capsize=7, color='green', linestyle=None, lw=1, markersize=7, elinewidth=1, label=f"GFP")

# violin_parts_50_50_MBP_post2 = plt.errorbar(3-0.25, MBP_mean_50_50_post2, yerr=MBP_std_50_50_post2, fmt='D', solid_capstyle='projecting', 
#                                    capsize=7, color='red', linestyle=None, lw=1, markersize=7, elinewidth=1, label=f"MBP")  
# violin_parts_50_50_GFP_post2 = plt.errorbar(3+0.25, GFP_mean_50_50_post2, yerr=GFP_std_50_50_post2, fmt='D', solid_capstyle='projecting', 
#                                capsize=7, color='green', linestyle=None, lw=1, markersize=7, elinewidth=1, label=f"GFP")

violin_parts_50_50_MBP_postcombined = plt.errorbar(5, MBP_mean_50_50_postcombined, yerr=MBP_std_50_50_postcombined, fmt='P', solid_capstyle='projecting', 
                                   capsize=7, color='red', linestyle=None, lw=1, markersize=7, elinewidth=1, label=f"MBP", alpha=0.33)  
violin_parts_50_50_GFP_postcombined = plt.errorbar(5, GFP_mean_50_50_postcombined, yerr=GFP_std_50_50_postcombined, fmt='P', solid_capstyle='projecting', 
                               capsize=7, color='green', linestyle=None, lw=1, markersize=7, elinewidth=1, label=f"GFP", alpha=0.33)
    

    
#vdir1_patch = mpatches.Patch(color='goldenrod', alpha=0.33, label=f"Direct 1 Psi Calls")#label=f"mu: {mean2}")
#vdir2_patch = mpatches.Patch(color='brown', alpha=0.33, label=f"Direct 2 Psi Calls")#label=f"mu: {mean2}")
#vdir3_patch = mpatches.Patch(color='purple', alpha=0.33, label=f"Direct 3 Psi Calls")#label=f"mu: {mean2}")
#v_IVT_patch = mpatches.Patch(color='r', alpha=0.33, label=f"IVT Psi Calls (FP)")#label=f"mu: {mean2}")

ax.legend(handles=[violin_parts_40_60_MBP, violin_parts_40_60_GFP], loc='upper left', prop={'size': 10})

#ax.legend(handles=[vdir1_patch, vdir2_patch, vdir3_patch], loc='upper right', prop={'size': 9})
#ax.legend(handles=[violin_parts_dir1_psi, violin_parts_dir2_psi, violin_parts_dir3_psi], loc='upper right', prop={'size': 12})

 
##### for idx in x_ticks[1:]:
#####     #print("..." + str(samp))
#####     plt.vlines(idx-0.5, 0, 1, color='k', linestyle='-.', lw=0.75 )
##### #plt.grid(axis="y")

# plt.savefig('MBPD10_vs_GFPD10_Ratios_Total_Updated_combined_pre_and_post_revs.png')#, transparent=True)
# plt.savefig('MBPD10_vs_GFPD10_Ratios_Total_Updated_combined_pre_and_post_revs.svg', transparent=True)


#save_results_to = 'Figures/Figure3/'
# plt.savefig(save_results_to + f'All_4_Dir_Results_D123.png', transparent=True, dpi = 300)
# plt.savefig(save_results_to + f'All_4_Dir_Results_D123.svg', transparent=True, dpi = 300)



In [ ]:
####Plotting mixture calls with probability scores####
import itertools
import seaborn as sns
from matplotlib import gridspec
import matplotlib as mpl
import joblib
import matplotlib.ticker as mticker
sns.set()
sns.set_style(style='white')

#%matplotlib qt5
%matplotlib inline

calls_dict = {'GFP10D': [], 'MBP10D': []}

#############################################################################
#filter_file = "B071222"

#MBP_GFP_mixed = nomixed.loc[nomixed['type']=='MBP+GFP10D_20_80']
#MBP_GFP_mixed = nomixed.loc[nomixed['type']=='MBP+GFP10D_40_60']
MBP_GFP_mixed = nomixed.loc[nomixed['type']=='MBP+GFP10D_50_50']
#MBP_GFP_mixed = nomixed.loc[nomixed['type']=='MBP+GFP10D_60_40']
#MBP_GFP_mixed = nomixed.loc[nomixed['type']=='MBP+GFP10D_80_20']

#MBP_GFP_mixed = MBP_GFP_mixed[MBP_GFP_mixed['file'].str.contains(filter_file)!=True]
print(f"Size of mixture dataframe we are predicting on: {len(MBP_GFP_mixed)}")
#############################################################################
nomixed=eventsDf
prior_NA = len(nomixed)

nomixed=nomixed.loc[nomixed['duration']>300e-6].loc[nomixed['duration']<20e-3] #minimum of 125 samples and maximum of 5000 samples
#nomixed=nomixed.loc[nomixed['duration']>900e-6]

post_NA = len(nomixed)
removed_NA_df = nomixed.dropna()

print(f"Size of dataframe prior to filtering by duration: {prior_NA}")
print(f"Size of dataframe after filtering by duration: {post_NA}")
print(f"Size of dataframe after  filtering by duration and dropping NaNs from dataframe: {len(removed_NA_df)}")

updated_df_list = []
type_list = ["MBP10D", "GFP10D"]
#type_list = ["MBP10D", "MBPMBP10D"]
#type_list = ["MBP10D", "MBP Nterm"]
#type_list = ["MBP Nterm", "MBPMBP10D"]
#type_list = ["MBP10D", "MBP Nterm", "MBPMBP10D"]

#Labels = ["MBPD10", "D10MBP", "diMBPD10"]
Labels = ["MBPD10", "GFPD10"]

for label in type_list:
    df = nomixed.loc[nomixed["type"] == label]
    updated_df_list.append(df)
noMixed = pd.concat(updated_df_list)
noMixed['type'].value_counts()
limiting_sample_size = min(noMixed['type'].value_counts())

print(f"Sample size of each class for training and testing: {limiting_sample_size}")
seedVal_sample = random.randint(0, 200)

updated_df_list = []
for label in type_list:
    df = noMixed.loc[noMixed["type"]==label].sample(limiting_sample_size, random_state=seedVal_sample)
    updated_df_list.append(df)
noMixed = pd.concat(updated_df_list)
    #noMixed['type'].value_counts()
print(f"Total size of training and testing data: {noMixed['type'].value_counts()}")


x=noMixed.drop(columns=['event','file','I_0','Imax', 'mean', 'std', 'event_start', 'event_end'])#, 'fc_0'])
x=x.drop(columns=['type','Blockage','mean/I_0', 'duration', 'std/I_0', 'Imin', 'max/std', 'min/duration', 'duration/std'])#'Imax/I_0','min/max','Imin/I_0','min/mean','min/std', 'mean/std'])
x=x.drop(columns=['min/max', 'min/mean', 'min/std', 'mean/std', 'Imin/I_0', 'Imax/I_0'])#'Imax/I_0','min/max','Imin/I_0','min/mean','min/std', 'mean/std'])

print(f"Feature space for training model: {feature_space}")
x = x.loc[:, feature_space]
y=noMixed['type']
seedVal_trte_split = random.randint(0, 200)
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.2, random_state=seedVal_trte_split) #2 #40 (123 seed was good for all four analytes)
scaler = sk.preprocessing.StandardScaler()
x_train = scaler.fit_transform( x_train )
x_test = scaler.transform( x_test )
    
print(f"Expected: {y_test.value_counts()}")
        
gbc = GradientBoostingClassifier(n_estimators=200,max_depth=5)
#rfc = RandomForestClassifier(n_estimators=200, max_depth=5)#, max_features=None)
#logisticRegr = LogisticRegression(solver = 'lbfgs')
#knn = KNeighborsClassifier(n_neighbors=5)
#clf = SVC(decision_function_shape='ovr',probability=True,kernel='rbf', 
           #C=200,verbose=True,gamma='scale', class_weight = None)

gbc.fit(x_train, y_train)

y_pred=gbc.predict(x_test)
unique, counts = np.unique(y_pred, return_counts=True)
print(f"Predicted: {dict(zip(unique, counts))}")

classification_accuracy.append(accuracy_score(y_test,y_pred)*100)
print('Mean classification accuracy of GBC model: {}'.format(np.mean(classification_accuracy)))  


###############################################################################################

mixed=MBP_GFP_mixed.drop(columns=['event','file','I_0','Imax', 'mean', 'std', 'event_start', 'event_end'])#, 'fc_0'])

#More features are removed
mixed = mixed.loc[:, feature_space]

mixed_scaled = scaler.transform(mixed)
mixed_pred = gbc.predict(mixed_scaled)
#mixed_exp_calls = gbc.predict(mixed)
#mixed_exp_calls = clf.predict(mixed_scaled)
unique, counts = np.unique(mixed_exp_calls, return_counts=True)
print(dict(zip(unique, counts)))

probability_scores = gbc.predict_proba(mixed_scaled)
##Probs = pd.DataFrame(probs_2, columns = ['GFP10D', 'MBP10D'])
Probs = pd.DataFrame(probability_scores, columns = ['GFP10D', 'MBP10D'])
print(Probs['GFP10D'])
print(Probs['MBP10D'])



#print(probability_scores)
#probs_2 = clf_2.predict_proba(X_norm_2)




#Prepare mixture dataset pandas dataframe
#mixed=eventsDf.loc[eventsDf['type']!="GFP10D"].loc[eventsDf['type']!="MBP10D"].loc[eventsDf['type']!="MBP Nterm"].loc[eventsDf['type']!="MBPMBP10D"].loc[eventsDf['type']!="GFP Dye 10D"]
##mixed=eventsDf.loc[eventsDf['type']!="GFP10D"].loc[eventsDf['type']!="MBP10D"]
##mixed=mixed.loc[mixed['duration']>500e-6].loc[mixed['duration']<20e-3]
##print(mixed['type'].value_counts())

#Remove features that were not used for SVM training, keeping the vectors of interest
##x_2=mixed.drop(columns=['type','event','file','I_0','Imax', 'mean', 'std', 'event_start', 'event_end'])
##x_2=x_2.drop(columns=['Blockage','mean/I_0', 'duration', 'std/I_0', 'Imin', 'max/std', 'min/duration', 'duration/std'])


#Normalize the data associated with each vector column-wise
#x_norm_2 = sk.preprocessing.StandardScaler().fit_transform(x_2)
#x_norm_2 = sk.preprocessing.MinMaxScaler(feature_range = (-1, 1)).fit_transform(x_2)
#X_norm_2 = pd.DataFrame(x_norm_2)

# scaler = sk.preprocessing.StandardScaler()
# x_norm_2 = scaler.fit_transform( x_2 )
# X_norm_2 = pd.DataFrame(x_norm_2)

##X_norm_2 = scaler.transform( x_2 )
#save trained model
##joblib.dump(clf, "MBP_GFP_Model.pkl") 

#load trained model and test on unlabeled mixture
##clf_2 = joblib.load("MBP_GFP_Model.pkl")

#Extract probability and decision function output from model predictions
##mixed_pred = clf_2.predict(X_norm_2)
#Let's observe the how many events were classified as GFP and MBP in the mixed experiments
##unique, counts = np.unique(mixed_pred, return_counts=True)
##print(dict(zip(unique, counts)))
##probs_2 = clf_2.predict_proba(X_norm_2)
##dec_2 = clf_2.decision_function(X_norm_2)



#Set up dataframe containing the probability confidence scores 
##Probs = pd.DataFrame(probability_scores, columns = ['GFP10D', 'MBP10D'])
#print(Probs['GFP10D'])
#print(Probs['MBP10D'])

#Load one of the mixed sample files for event classification
##file = experiments["MBP+GFP10D"][1]

#####################################################################################################
#file = experiments["MBP+GFP10D_80_20"][10]
#file = experiments["MBP+GFP10D_20_80"][0]
#file = experiments["MBP+GFP10D_50_50"][25]
file = experiments["MBP+GFP10D_50_50"][28]
#####################################################################################################

#Let's make a new dataframe that now has the classifications associated with each event
mixed_classified = mixed
mixed_classified ['type'] = mixed_pred
mixed_classified ["pGFP"] = probability_scores[:,0]
mixed_classified ["pMBP"] = probability_scores[:,1]
mixed_classified ["file"] = MBP_GFP_mixed['file']
mixed_classified ["event_start"] = MBP_GFP_mixed['event_start']

#Plot the decsion function
##plt.figure(1)
##plt.plot(dec_2[mixed_pred=='GFP10D'], marker="o", c='g')
##plt.plot(dec_2[mixed_pred=='MBP10D'], marker="o", c='r')

#Plot the decsion function with respect to probability score
##plt.figure(2)
##plt.scatter(dec_2[mixed_pred=='GFP10D'],probs_2[:,0][mixed_pred=='GFP10D'],c='g')
##plt.scatter(dec_2[mixed_pred=='MBP10D'],probs_2[:,0][mixed_pred=='MBP10D'],c='r')

#Plotting function for classifying events in a particular mixed experiment file
plt.figure(3)
barsGFP = []
barsMBP = []
x_pos=[]
widths=[]

einfo=[]
for event in file.events:
    #event.classification=None
    event.classification=2 #Set events that did not get trained in the SVM to a black color
    c=mixed_classified.loc[(mixed['event_start']==event.start) & 
                           (file.filename==mixed['file'])] #& 
    
    if not c.empty:
        x_pos.append(event.start+((event.end-event.start)/2))
        widths.append(1)#(event.end-event.start)
        barsGFP.append(c.iloc[0]['pGFP'])
        barsMBP.append(c.iloc[0]['pMBP'])
        
        
        if(c.iloc[0]['type']=='GFP10D'): 
            event.classification=0
            einfo.append((int(event.second*(event.start-10e-4)),
                      int(event.second*(event.end+10e-4)),event.classification)) 
                     
        elif (c.iloc[0]['type']=='MBP10D'): 
            event.classification=1
            einfo.append((int(event.second*(event.start-10e-4)),
                      int(event.second*(event.end+10e-4)),event.classification)) 
                
m_file = mixed_classified.loc[(mixed_classified['file']==file.filename)] 
m_file ["x_pos"] = x_pos


####Function for probability y-axis tics###
def update_ticks(y, pos):
    if y == 0.0:
        return '100% GFP'
    elif y == 1.0:
        return '100% MBP'
    else:
        return y

from matplotlib.colors import LinearSegmentedColormap
import matplotlib.colors as mcolors
mpl.rcParams['lines.linewidth'] = 1.5

###Making weight column for probability score###
color_weights = []
for index, row in m_file.iterrows():
    #print(row['pMBP'])
    color_weights.append(abs(row['pMBP']-0.5))
m_file ["color_weights"] = color_weights

#Plot colored events with probability scores associated with each event and gradient transparency associated with prob score
# ax2=plt.subplot(gs[-1, :],sharex=ax1)
# x = m_file.loc[m_file["type"]=='GFP10D']["x_pos"]
# y = m_file.loc[m_file["type"]=='GFP10D']["pMBP"]
# z =  m_file.loc[m_file["type"]=='GFP10D']["color_weights"]
# plt.figure(figsize=(21,9),dpi=200)
# plt.scatter(x, y, s=200, c=z, cmap='Greens', edgecolors='k')
# x = m_file.loc[m_file["type"]=='MBP10D']["x_pos"]
# y = m_file.loc[m_file["type"]=='MBP10D']["pMBP"]
# z =  m_file.loc[m_file["type"]=='MBP10D']["color_weights"]
# plt.scatter(x, y, s=200, c=z, cmap='Reds', edgecolors='k')
# ax2.set(xlabel='Time [s]', ylabel='Decision Probability')
# plt.yticks([0.0, 0.5, 1.0])
# plt.axhline(y = 0.5, color = 'k', linestyle = 'dashed')
# plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
# plt.xlim(20, 145)
# ax2.yaxis.set_major_formatter(mticker.FuncFormatter(update_ticks))

#Plot colored events with probability scores associated with each event and gradient transparency associated with prob score
plt.figure(figsize=(21,14),dpi=300)
gs = gridspec.GridSpec(4,1) 
ax1=plt.subplot(gs[:3, :]) 
file.plot( limits=None, color_events=True, event_downsample=1, 
        file_downsample=10, downsample=10, file_kwargs={ 'c':'k', 'alpha':1},
        event_kwargs={ 'c':'c', 'alpha':1 }, multiclass=True,classcolors=['g','r', 'black', 'purple'])
ax1.set_xlabel('', fontsize = 20)
ax1.set_ylabel('Ionic Current [nA]', fontsize = 20)
#ax1.set(title= 'MBP and GFP Mixture GBC Model Classification',xlabel='Time [s]', ylabel='Ionic Current [nA]')    

ax2=plt.subplot(gs[-1, :],sharex=ax1)
x1 = m_file.loc[m_file["type"]=='GFP10D']["x_pos"]
y1 = m_file.loc[m_file["type"]=='GFP10D']["pMBP"]
z1 =  m_file.loc[m_file["type"]=='GFP10D']["color_weights"]
plt.scatter(x1, y1, s=200, c=z1, cmap='Greens', edgecolors='k')
x2 = m_file.loc[m_file["type"]=='MBP10D']["x_pos"]
y2 = m_file.loc[m_file["type"]=='MBP10D']["pMBP"]
z2 =  m_file.loc[m_file["type"]=='MBP10D']["color_weights"]
plt.scatter(x2, y2, s=200, c=z2, cmap='Reds', edgecolors='k')
#ax2.set(xlabel='Time [s]', ylabel='Decision Probability')
ax2.set_xlabel('Time [s]', fontsize = 20)
ax2.set_ylabel('Decision Probability', fontsize = 20)

ax2.set_ylim([-0.1, 1.1])
ax2.yaxis.set_major_formatter(mticker.FuncFormatter(update_ticks))
plt.yticks([0.0, 0.5, 1.0])
plt.axhline(y = 0.5, color = 'k', linestyle = 'dashed')
plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)

#plt.xlim(15.5, 32.5) #80 to 20 limit
#plt.xlim(1, 18) #20 to 80 limit
plt.xlim(38, 55.5) #50 to 50 limit

#plt.savefig('Unlabeled_Calls_Prob_50_50.png', dpi = 300)
#plt.savefig('Unlabeled_Calls_Prob_50_50.svg', transparent=True, dpi = 300)    







###Plot the matrix with a heatmap colorbar
plt.figure()
data = np.random.rand(10,10) * 2 - 1
colors1 = plt.cm.Greens(np.linspace(1., 0., 128, endpoint=True))
colors2 = plt.cm.Reds(np.linspace(0., 1., 128, endpoint=True))
#combine them and build a new colormap
colors = np.vstack((colors1, colors2))
mymap = mcolors.LinearSegmentedColormap.from_list('my_colormap', colors)

v1 = np.linspace(0, 1.0, 5, endpoint=True)
#plt.pcolor(data, cmap=mymap)
cbar=plt.imshow(data, vmin=-1., vmax=1., cmap=mymap)
cbar = plt.colorbar()
#plt.savefig("Color_Map.svg", transparent=True)



###With seaborn###
# ax2=plt.subplot(gs[-1, :],sharex=ax1)
# sns.scatterplot(data=m_file, x="x_pos", y="pMBP", hue="type", marker="o", palette=['red','green'], s=120)
# ax2.set(xlabel='Time [s]', ylabel='Decision Probability')
# plt.yticks([0.0, 0.5, 1.0])
# plt.axhline(y = 0.5, color = 'k', linestyle = 'dashed')
# plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
# plt.xlim(20, 145)
# ax2.yaxis.set_major_formatter(mticker.FuncFormatter(update_ticks))
# #plt.savefig("Probability_Scored_Events.svg", transparent=True)


#Only plots the colored events
plt.figure(figsize=(1,1))
file.plot( limits=None, color_events=True, event_downsample=1, 
        file_downsample=10, downsample=10, file_kwargs={ 'c':'k', 'alpha':1},
        event_kwargs={ 'c':'c', 'alpha':1 }, multiclass=True, classcolors=['g','r', 'black', 'purple'])  
plt.xlabel('Time [s]')
plt.ylabel('Ionic Current [nA]')
plt.title('MBP and GFP Mixture SVM Classification')
plt.xlim(20, 30)


#print(x_pos)
#print(einfo)
#print("Here: " + str(m_file["pMBP"]))











In [ ]:
####Plotting mixture calls with probability scores####
import itertools
import seaborn as sns
from matplotlib import gridspec
import matplotlib as mpl
import joblib
import matplotlib.ticker as mticker
sns.set()
sns.set_style(style='white')

#%matplotlib qt5
%matplotlib inline

calls_dict = {'GFP10D': [], 'MBP10D': []}

#############################################################################
#filter_file = "B071222"

#MBP_GFP_mixed = nomixed.loc[nomixed['type']=='MBP+GFP10D_20_80']
#MBP_GFP_mixed = nomixed.loc[nomixed['type']=='MBP+GFP10D_40_60']
MBP_GFP_mixed = nomixed.loc[nomixed['type']=='MBP+GFP10D_50_50']
#MBP_GFP_mixed = nomixed.loc[nomixed['type']=='MBP+GFP10D_60_40']
#MBP_GFP_mixed = nomixed.loc[nomixed['type']=='MBP+GFP10D_80_20']

#MBP_GFP_mixed = MBP_GFP_mixed[MBP_GFP_mixed['file'].str.contains(filter_file)!=True]
print(f"Size of mixture dataframe we are predicting on: {len(MBP_GFP_mixed)}")
#############################################################################
nomixed=eventsDf
prior_NA = len(nomixed)

nomixed=nomixed.loc[nomixed['duration']>300e-6].loc[nomixed['duration']<20e-3] #minimum of 125 samples and maximum of 5000 samples
#nomixed=nomixed.loc[nomixed['duration']>900e-6]

post_NA = len(nomixed)
removed_NA_df = nomixed.dropna()

print(f"Size of dataframe prior to filtering by duration: {prior_NA}")
print(f"Size of dataframe after filtering by duration: {post_NA}")
print(f"Size of dataframe after  filtering by duration and dropping NaNs from dataframe: {len(removed_NA_df)}")

updated_df_list = []
type_list = ["MBP10D", "GFP10D"]
#type_list = ["MBP10D", "MBPMBP10D"]
#type_list = ["MBP10D", "MBP Nterm"]
#type_list = ["MBP Nterm", "MBPMBP10D"]
#type_list = ["MBP10D", "MBP Nterm", "MBPMBP10D"]

#Labels = ["MBPD10", "D10MBP", "diMBPD10"]
Labels = ["MBPD10", "GFPD10"]

for label in type_list:
    df = nomixed.loc[nomixed["type"] == label]
    updated_df_list.append(df)
noMixed = pd.concat(updated_df_list)
noMixed['type'].value_counts()
limiting_sample_size = min(noMixed['type'].value_counts())

print(f"Sample size of each class for training and testing: {limiting_sample_size}")
seedVal_sample = random.randint(0, 200)

updated_df_list = []
for label in type_list:
    df = noMixed.loc[noMixed["type"]==label].sample(limiting_sample_size, random_state=seedVal_sample)
    updated_df_list.append(df)
noMixed = pd.concat(updated_df_list)
    #noMixed['type'].value_counts()
print(f"Total size of training and testing data: {noMixed['type'].value_counts()}")


x=noMixed.drop(columns=['event','file','I_0','Imax', 'mean', 'std', 'event_start', 'event_end'])#, 'fc_0'])
x=x.drop(columns=['type','Blockage','mean/I_0', 'duration', 'std/I_0', 'Imin', 'max/std', 'min/duration', 'duration/std'])#'Imax/I_0','min/max','Imin/I_0','min/mean','min/std', 'mean/std'])
x=x.drop(columns=['min/max', 'min/mean', 'min/std', 'mean/std', 'Imin/I_0', 'Imax/I_0'])#'Imax/I_0','min/max','Imin/I_0','min/mean','min/std', 'mean/std'])

print(f"Feature space for training model: {feature_space}")
x = x.loc[:, feature_space]
y=noMixed['type']
seedVal_trte_split = random.randint(0, 200)
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.2, random_state=seedVal_trte_split) #2 #40 (123 seed was good for all four analytes)
scaler = sk.preprocessing.StandardScaler()
x_train = scaler.fit_transform( x_train )
x_test = scaler.transform( x_test )
    
print(f"Expected: {y_test.value_counts()}")
        
gbc = GradientBoostingClassifier(n_estimators=200,max_depth=5)
#rfc = RandomForestClassifier(n_estimators=200, max_depth=5)#, max_features=None)
#logisticRegr = LogisticRegression(solver = 'lbfgs')
#knn = KNeighborsClassifier(n_neighbors=5)
#clf = SVC(decision_function_shape='ovr',probability=True,kernel='rbf', 
           #C=200,verbose=True,gamma='scale', class_weight = None)

gbc.fit(x_train, y_train)

y_pred=gbc.predict(x_test)
unique, counts = np.unique(y_pred, return_counts=True)
print(f"Predicted: {dict(zip(unique, counts))}")

classification_accuracy.append(accuracy_score(y_test,y_pred)*100)
print('Mean classification accuracy of GBC model: {}'.format(np.mean(classification_accuracy)))  


###############################################################################################

mixed=MBP_GFP_mixed.drop(columns=['event','file','I_0','Imax', 'mean', 'std', 'event_start', 'event_end'])#, 'fc_0'])

#More features are removed
mixed = mixed.loc[:, feature_space]

mixed_scaled = scaler.transform(mixed)
mixed_pred = gbc.predict(mixed_scaled)
#mixed_exp_calls = gbc.predict(mixed)
#mixed_exp_calls = clf.predict(mixed_scaled)
unique, counts = np.unique(mixed_exp_calls, return_counts=True)
print(dict(zip(unique, counts)))

probability_scores = gbc.predict_proba(mixed_scaled)
##Probs = pd.DataFrame(probs_2, columns = ['GFP10D', 'MBP10D'])
Probs = pd.DataFrame(probability_scores, columns = ['GFP10D', 'MBP10D'])
print(Probs['GFP10D'])
print(Probs['MBP10D'])



#print(probability_scores)
#probs_2 = clf_2.predict_proba(X_norm_2)




#Prepare mixture dataset pandas dataframe
#mixed=eventsDf.loc[eventsDf['type']!="GFP10D"].loc[eventsDf['type']!="MBP10D"].loc[eventsDf['type']!="MBP Nterm"].loc[eventsDf['type']!="MBPMBP10D"].loc[eventsDf['type']!="GFP Dye 10D"]
##mixed=eventsDf.loc[eventsDf['type']!="GFP10D"].loc[eventsDf['type']!="MBP10D"]
##mixed=mixed.loc[mixed['duration']>500e-6].loc[mixed['duration']<20e-3]
##print(mixed['type'].value_counts())

#Remove features that were not used for SVM training, keeping the vectors of interest
##x_2=mixed.drop(columns=['type','event','file','I_0','Imax', 'mean', 'std', 'event_start', 'event_end'])
##x_2=x_2.drop(columns=['Blockage','mean/I_0', 'duration', 'std/I_0', 'Imin', 'max/std', 'min/duration', 'duration/std'])


#Normalize the data associated with each vector column-wise
#x_norm_2 = sk.preprocessing.StandardScaler().fit_transform(x_2)
#x_norm_2 = sk.preprocessing.MinMaxScaler(feature_range = (-1, 1)).fit_transform(x_2)
#X_norm_2 = pd.DataFrame(x_norm_2)

# scaler = sk.preprocessing.StandardScaler()
# x_norm_2 = scaler.fit_transform( x_2 )
# X_norm_2 = pd.DataFrame(x_norm_2)

##X_norm_2 = scaler.transform( x_2 )
#save trained model
##joblib.dump(clf, "MBP_GFP_Model.pkl") 

#load trained model and test on unlabeled mixture
##clf_2 = joblib.load("MBP_GFP_Model.pkl")

#Extract probability and decision function output from model predictions
##mixed_pred = clf_2.predict(X_norm_2)
#Let's observe the how many events were classified as GFP and MBP in the mixed experiments
##unique, counts = np.unique(mixed_pred, return_counts=True)
##print(dict(zip(unique, counts)))
##probs_2 = clf_2.predict_proba(X_norm_2)
##dec_2 = clf_2.decision_function(X_norm_2)



#Set up dataframe containing the probability confidence scores 
##Probs = pd.DataFrame(probability_scores, columns = ['GFP10D', 'MBP10D'])
#print(Probs['GFP10D'])
#print(Probs['MBP10D'])

#Load one of the mixed sample files for event classification
##file = experiments["MBP+GFP10D"][1]

#####################################################################################################
#file = experiments["MBP+GFP10D_80_20"][10]
#file = experiments["MBP+GFP10D_20_80"][0]
#file = experiments["MBP+GFP10D_50_50"][25]
file = experiments["MBP+GFP10D_50_50"][28]
#####################################################################################################

#Let's make a new dataframe that now has the classifications associated with each event
mixed_classified = mixed
mixed_classified ['type'] = mixed_pred
mixed_classified ["pGFP"] = probability_scores[:,0]
mixed_classified ["pMBP"] = probability_scores[:,1]
mixed_classified ["file"] = MBP_GFP_mixed['file']
mixed_classified ["event_start"] = MBP_GFP_mixed['event_start']

#Plot the decsion function
##plt.figure(1)
##plt.plot(dec_2[mixed_pred=='GFP10D'], marker="o", c='g')
##plt.plot(dec_2[mixed_pred=='MBP10D'], marker="o", c='r')

#Plot the decsion function with respect to probability score
##plt.figure(2)
##plt.scatter(dec_2[mixed_pred=='GFP10D'],probs_2[:,0][mixed_pred=='GFP10D'],c='g')
##plt.scatter(dec_2[mixed_pred=='MBP10D'],probs_2[:,0][mixed_pred=='MBP10D'],c='r')

#Plotting function for classifying events in a particular mixed experiment file
plt.figure(3)
barsGFP = []
barsMBP = []
x_pos=[]
widths=[]

einfo=[]
for event in file.events:
    #event.classification=None
    event.classification=2 #Set events that did not get trained in the SVM to a black color
    c=mixed_classified.loc[(mixed['event_start']==event.start) & 
                           (file.filename==mixed['file'])] #& 
    
    if not c.empty:
        x_pos.append(event.start+((event.end-event.start)/2))
        widths.append(1)#(event.end-event.start)
        barsGFP.append(c.iloc[0]['pGFP'])
        barsMBP.append(c.iloc[0]['pMBP'])
        
        
        if(c.iloc[0]['type']=='GFP10D'): 
            event.classification=0
            einfo.append((int(event.second*(event.start-10e-4)),
                      int(event.second*(event.end+10e-4)),event.classification)) 
                     
        elif (c.iloc[0]['type']=='MBP10D'): 
            event.classification=1
            einfo.append((int(event.second*(event.start-10e-4)),
                      int(event.second*(event.end+10e-4)),event.classification)) 
                
m_file = mixed_classified.loc[(mixed_classified['file']==file.filename)] 
m_file ["x_pos"] = x_pos


####Function for probability y-axis tics###
def update_ticks(y, pos):
    if y == 0.0:
        return '100% GFP'
    elif y == 1.0:
        return '100% MBP'
    else:
        return y

from matplotlib.colors import LinearSegmentedColormap
import matplotlib.colors as mcolors
mpl.rcParams['lines.linewidth'] = 1.5

###Making weight column for probability score###
color_weights = []
for index, row in m_file.iterrows():
    #print(row['pMBP'])
    color_weights.append(abs(row['pMBP']-0.5))
m_file ["color_weights"] = color_weights

#Plot colored events with probability scores associated with each event and gradient transparency associated with prob score
# ax2=plt.subplot(gs[-1, :],sharex=ax1)
# x = m_file.loc[m_file["type"]=='GFP10D']["x_pos"]
# y = m_file.loc[m_file["type"]=='GFP10D']["pMBP"]
# z =  m_file.loc[m_file["type"]=='GFP10D']["color_weights"]
# plt.figure(figsize=(21,9),dpi=200)
# plt.scatter(x, y, s=200, c=z, cmap='Greens', edgecolors='k')
# x = m_file.loc[m_file["type"]=='MBP10D']["x_pos"]
# y = m_file.loc[m_file["type"]=='MBP10D']["pMBP"]
# z =  m_file.loc[m_file["type"]=='MBP10D']["color_weights"]
# plt.scatter(x, y, s=200, c=z, cmap='Reds', edgecolors='k')
# ax2.set(xlabel='Time [s]', ylabel='Decision Probability')
# plt.yticks([0.0, 0.5, 1.0])
# plt.axhline(y = 0.5, color = 'k', linestyle = 'dashed')
# plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
# plt.xlim(20, 145)
# ax2.yaxis.set_major_formatter(mticker.FuncFormatter(update_ticks))

#Plot colored events with probability scores associated with each event and gradient transparency associated with prob score
plt.figure(figsize=(21,14),dpi=300)
gs = gridspec.GridSpec(4,1) 
ax1=plt.subplot(gs[:3, :]) 
file.plot( limits=None, color_events=True, event_downsample=1, 
        file_downsample=10, downsample=10, file_kwargs={ 'c':'k', 'alpha':1},
        event_kwargs={ 'c':'c', 'alpha':1 }, multiclass=True,classcolors=['g','r', 'black', 'purple'])
ax1.set_xlabel('', fontsize = 20)
ax1.set_ylabel('Ionic Current [nA]', fontsize = 20)
#ax1.set(title= 'MBP and GFP Mixture GBC Model Classification',xlabel='Time [s]', ylabel='Ionic Current [nA]')    

ax2=plt.subplot(gs[-1, :],sharex=ax1)
x1 = m_file.loc[m_file["type"]=='GFP10D']["x_pos"]
y1 = m_file.loc[m_file["type"]=='GFP10D']["pMBP"]
z1 =  m_file.loc[m_file["type"]=='GFP10D']["color_weights"]
plt.scatter(x1, y1, s=200, c=z1, cmap='Greens', edgecolors='k')
x2 = m_file.loc[m_file["type"]=='MBP10D']["x_pos"]
y2 = m_file.loc[m_file["type"]=='MBP10D']["pMBP"]
z2 =  m_file.loc[m_file["type"]=='MBP10D']["color_weights"]
plt.scatter(x2, y2, s=200, c=z2, cmap='Reds', edgecolors='k')
#ax2.set(xlabel='Time [s]', ylabel='Decision Probability')
ax2.set_xlabel('Time [s]', fontsize = 20)
ax2.set_ylabel('Decision Probability', fontsize = 20)

ax2.set_ylim([-0.1, 1.1])
ax2.yaxis.set_major_formatter(mticker.FuncFormatter(update_ticks))
plt.yticks([0.0, 0.5, 1.0])
plt.axhline(y = 0.5, color = 'k', linestyle = 'dashed')
plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)

#plt.xlim(15.5, 32.5) #80 to 20 limit
#plt.xlim(1, 18) #20 to 80 limit
plt.xlim(38, 55.5) #50 to 50 limit

#plt.savefig('Unlabeled_Calls_Prob_50_50.png', dpi = 300)
#plt.savefig('Unlabeled_Calls_Prob_50_50.svg', transparent=True, dpi = 300)    




###Plot the matrix with a heatmap colorbar
plt.figure()
data = np.random.rand(10,10) * 2 - 1
colors1 = plt.cm.Greens(np.linspace(1., 0., 128, endpoint=True))
colors2 = plt.cm.Reds(np.linspace(0., 1., 128, endpoint=True))
#combine them and build a new colormap
colors = np.vstack((colors1, colors2))
mymap = mcolors.LinearSegmentedColormap.from_list('my_colormap', colors)

v1 = np.linspace(0, 1.0, 5, endpoint=True)
#plt.pcolor(data, cmap=mymap)
cbar=plt.imshow(data, vmin=-1., vmax=1., cmap=mymap)
cbar = plt.colorbar()
#plt.savefig("Color_Map.svg", transparent=True)



###With seaborn###
# ax2=plt.subplot(gs[-1, :],sharex=ax1)
# sns.scatterplot(data=m_file, x="x_pos", y="pMBP", hue="type", marker="o", palette=['red','green'], s=120)
# ax2.set(xlabel='Time [s]', ylabel='Decision Probability')
# plt.yticks([0.0, 0.5, 1.0])
# plt.axhline(y = 0.5, color = 'k', linestyle = 'dashed')
# plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
# plt.xlim(20, 145)
# ax2.yaxis.set_major_formatter(mticker.FuncFormatter(update_ticks))
# #plt.savefig("Probability_Scored_Events.svg", transparent=True)


#Only plots the colored events
plt.figure(figsize=(1,1))
file.plot( limits=None, color_events=True, event_downsample=1, 
        file_downsample=10, downsample=10, file_kwargs={ 'c':'k', 'alpha':1},
        event_kwargs={ 'c':'c', 'alpha':1 }, multiclass=True, classcolors=['g','r', 'black', 'purple'])  
plt.xlabel('Time [s]')
plt.ylabel('Ionic Current [nA]')
plt.title('MBP and GFP Mixture SVM Classification')
plt.xlim(20, 30)


#print(x_pos)
#print(einfo)
#print("Here: " + str(m_file["pMBP"]))



In [ ]:
#Grabbing and displaying 20 events, 10 that were called as GFPD10 and 10 as MBPD10
current=file.current


gfp=[e for e in einfo if e[2]==0]
mbp=[e for e in einfo if e[2]==1]
step=file.second
laststart = 0
gap=250

plt.figure(figsize=(20,12),dpi=200)
ax=plt.subplot(211)
plt.ylim(-50,400)
plt.title("Unlabeled MBPD10 Calls")
plt.ylabel("I [pA]")
for (start,end,_) in mbp[55:65]:
    print(start, end, _)
    thisend=laststart+(end-start)
    plt.plot (np.arange(laststart,thisend)/step,1000*current[start:end],linewidth=0.6, c='r', alpha=1)
    laststart=thisend+gap

laststart=0
plt.subplot(212,sharex=ax)
plt.ylim(-50,400)
plt.title ("Unlabeled GFPD10 Calls")
plt.ylabel("I [pA]")
for (start,end,_) in gfp[25:42]:
    thisend=laststart+(end-start)
    plt.plot (np.arange(laststart,thisend)/step,1000*current[start:end],linewidth=0.6, c='g', alpha=1)
    laststart=thisend+gap